# PROJETO FINAL: ANÁLISE DE RECLAMAÇÕES FINANCEIRAS - AGIBANK

**Trilha:** Dados - Agibank  
**Data:** Fevereiro 2025  
**Fonte:** Consumidor.gov.br (Senacon/MJSP)

---

## CONTEXTO DO PROJETO

Este projeto analisa **2.5+ milhões de reclamações** do Consumidor.gov.br (2025), com foco estratégico no **setor financeiro brasileiro**, abrangendo:

- 🏦 **Setor Financeiro Nacional** (bancos, fintechs, cartões de crédito)
- 🇧🇷 **Brasil** (panorama geral do mercado financeiro)
- 🏙️ **São Paulo** (principal hub financeiro do país)
- 🏢 **Agibank** (análise competitiva e posicionamento)

---

## OBJETIVOS

### Análise do Setor Financeiro
1. Mapear principais problemas reportados por consumidores
2. Identificar tendências e padrões de reclamações
3. Analisar taxas de resolução e tempo de resposta do setor
4. Comparar performance entre instituições financeiras

---

## ESCOPO DAS ANÁLISES

### Análises Quantitativas
- Volume e distribuição temporal de reclamações
- Segmentação por tipo de produto financeiro
- Métricas de satisfação e resolução
- Análise geográfica (foco em São Paulo)

### Análises Qualitativas
- Categorização de problemas recorrentes
- Análise de sentimento (quando aplicável)
- Identificação de gargalos operacionais
- Benchmarking competitivo

### Insights Estratégicos
- Oportunidades de melhoria no atendimento
- Produtos/serviços com maior índice de reclamações
- Recomendações para redução de churn
- Sugestões de expansão baseadas em gaps do mercado

---

## REQUISITOS ATENDIDOS

✅ **A. Higiene dos Dados:** Importação, limpeza e tratamento  
✅ **B. Performance (NumPy):** Vetorização e classificações complexas  
✅ **C. Análise Exploratória:** GroupBy e métricas estatísticas  
✅ **D. Visualização:** Gráficos com storytelling  

---

## DADOS

- **Fonte:** Consumidor.gov.br (Senacon/MJSP)
- **Período:** 2025
- **Volume Total:** 2.567.095 registros
- **Foco:** Setor Financeiro (bancos, fintechs, cartões, seguros)
- **Camadas:** Bronze → Silver → Gold
- **Granularidade:** Nacional, Regional (SP) e Empresa (Agibank)

In [2]:
# Importação de bibliotecas essenciais para análise de dados, visualização e manipulação de arquivos, garantindo que todas as ferramentas necessárias estejam disponíveis para realizar análises secundárias de forma eficiente e eficaz.

import sys
from pathlib import Path

# Adicionar pasta raiz ao path (3 níveis acima) --> garantir que os módulos customizados localizados na pasta 'lib' possam ser importados corretamente, facilitando a organização do código e a reutilização de funções e classes em diferentes partes do projeto.

sys.path.append('../../..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import plotly.express as px        
import plotly.graph_objects as go  
import altair as alt               
from dash import Dash, dcc, html   
import warnings
from datetime import datetime

# Configurações do projeto --> garantir que as análises sejam realizadas com base em um ano específico, e que os caminhos para salvar os resultados estejam definidos de forma clara e organizada, facilitando a gestão dos arquivos gerados durante o processo de análise.

# Módulo de carregamento de dados --> garantir que as bases de dados necessárias para as análises secundárias sejam carregadas de forma eficiente e consistente, utilizando funções customizadas que facilitam a leitura e manipulação dos dados, além de fornecer informações sobre os arquivos disponíveis e a estrutura das bases carregadas.
# Lib --> carregamento de dados, organização e informações sobre as bases de dados, garantindo que os dados sejam acessados de forma eficiente e estruturada para as análises subsequentes.

from lib.carregamento import (
    carregar_base_silver,
    carregar_base_gold_sp,
    carregar_base_agibank,
    carregar_base_filtrada,
    listar_arquivos_disponiveis,
    info_base
)

from lib.cores import (
    aplicar_tema_agibank,
    configurar_plotly,
    CORES_AGIBANK,
    PALETA_CATEGORICA,
    PALETA_AZUL,
    PALETA_VERDE,
    PLOTLY_COLORS,
    PLOTLY_PALETTE,
    PLOTLY_SCALE_AZUL,
    PLOTLY_SCALE_VERDE
)

from lib.visualizacoes import (
    grafico_barras,
    grafico_linha,
    grafico_pizza,
    grafico_boxplot,
    grafico_heatmap,
    grafico_distribuicao,
    grafico_comparativo_barras
)

print("✅ Todos os módulos importados com sucesso!")



Tema Agibank aplicado - Tamanho: medio
✅ Todos os módulos importados com sucesso!


In [3]:
# Ignorar alertas desnecessários
warnings.filterwarnings('ignore')

# Configurações de semente aleatória do NumPy (Reprodutibilidade)
np.random.seed(42)

# Configurações de exibição do NumPy
np.set_printoptions(
    precision=2,        # Casas decimais
    suppress=True,      # Evitar notação científica (1e-4)
    linewidth=120,      # Largura da linha no console
    edgeitems=5,        # Itens mostrados no início/fim de arrays longos
    threshold=1000      # Limite para resumir arrays grandes
)

# Configurações de exibição do Pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', '{:.2f}'.format)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 50)

# Integração Pandas-NumPy (Opcional: garante compatibilidade de tipos)
pd.set_option('mode.use_inf_as_na', True)

# Aplicar temas visuais
aplicar_tema_agibank('grande')  # Matplotlib/Seaborn
configurar_plotly()              # Plotly

# Configurações do Altair
alt.renderers.enable('default')
alt.data_transformers.disable_max_rows()

# Configurações para gráficos inline no Jupyter
try:
    get_ipython().run_line_magic('matplotlib', 'inline')
except:
    pass

# Configurações de salvamento de figuras
plt.rcParams['savefig.dpi'] = 300
plt.rcParams['savefig.bbox'] = 'tight'

Tema Agibank aplicado - Tamanho: grande
Tema Agibank aplicado ao Plotly


In [4]:
# Definição de constantes temporais com NumPy
ANO_ANALISE = np.int16(2025)
JANELA_ANOS = np.array([ANO_ANALISE - 1, ANO_ANALISE], dtype=np.int16) # Comparativo YoY
DATAS_REFERENCIA = np.arange(f'{ANO_ANALISE}-01', f'{ANO_ANALISE+1}-01', dtype='datetime64[M]')

# Definição de caminhos do projeto
RAIZ_PROJETO = Path.cwd()
CAMINHO_OUTPUT = RAIZ_PROJETO / 'output'
CAMINHO_OUTPUT.mkdir(exist_ok=True)

# Feedback do ambiente
print(f"✅ Configurações iniciais concluídas!")
print(f"Ambiente pronto para análise do setor financeiro em {ANO_ANALISE}")
print(f"Janela de análise definida: {JANELA_ANOS}")
print(f"Horizonte temporal (meses): {len(DATAS_REFERENCIA)} períodos")

✅ Configurações iniciais concluídas!
Ambiente pronto para análise do setor financeiro em 2025
Janela de análise definida: [2024 2025]
Horizonte temporal (meses): 12 períodos


In [5]:
# Cabeçalho de sucesso
print("\n" + "=" * 80)
print("AMBIENTE CONFIGURADO COM SUCESSO")
print("=" * 80)

# Obter versão do Plotly com tratamento de erro
try:
    plotly_v = plotly.__version__
except:
    plotly_v = "Instalado"

# Estruturar metadados das bibliotecas em um Array NumPy (Matriz de Strings)
info_libs = np.array([
    ['Pandas', pd.__version__],
    ['NumPy', np.__version__],
    ['Matplotlib', plt.matplotlib.__version__],
    ['Seaborn', sns.__version__],
    ['Plotly', plotly_v],
    ['Altair', alt.__version__],
    ['Python', sys.version.split()[0]]
])

print(f"\nVersoes das bibliotecas:")
# Iterar sobre o array NumPy para exibição limpa
for lib, versao in info_libs:
    print(f"  {lib}: {versao}")

print(f"\nConfiguracoes do projeto:")
print(f"  Ano de analise: {ANO_ANALISE}")
print(f"  Pasta de output: {CAMINHO_OUTPUT}")
print(f"  Path atual: {RAIZ_PROJETO}")

# Converter paletas para NumPy para análise de tamanho
paleta_mpl_np = np.array(PALETA_CATEGORICA)
paleta_plotly_np = np.array(PLOTLY_PALETTE)

print(f"\nTema visual:")
print(f"  Tema Agibank aplicado (tamanho: grande)")
# Uso de .size (propriedade NumPy) ao invés de len()
print(f"  Paleta Matplotlib: {paleta_mpl_np.size} cores")
print(f"  Paleta Plotly: {paleta_plotly_np.size} cores")

print("=" * 80)


AMBIENTE CONFIGURADO COM SUCESSO

Versoes das bibliotecas:
  Pandas: 2.3.3
  NumPy: 2.3.5
  Matplotlib: 3.10.7
  Seaborn: 0.13.2
  Plotly: Instalado
  Altair: 6.0.0
  Python: 3.13.5

Configuracoes do projeto:
  Ano de analise: 2025
  Pasta de output: c:\Users\caroline.coutinho\projeto_mediacao_bancaria\analises\silver\notebooks_silver\output
  Path atual: c:\Users\caroline.coutinho\projeto_mediacao_bancaria\analises\silver\notebooks_silver

Tema visual:
  Tema Agibank aplicado (tamanho: grande)
  Paleta Matplotlib: 6 cores
  Paleta Plotly: 6 cores


In [6]:
# Filtrar dados do setor financeiro para o ano de análise, garantindo que as análises subsequentes sejam realizadas com base em um conjunto de dados relevante e específico para o período de interesse, permitindo a identificação de tendências sazonais, períodos de maior ou menor atividade e o impacto das reclamações no setor financeiro ao longo do ano.

separador = str(np.char.multiply('=', 80))

# Definição das mensagens em um array NumPy para estruturação

mensagens_header = np.array([
    "\n" + separador,
    "ANÁLISES DO SETOR FINANCEIRO - BRASIL",
    separador,
    "\nEscopo: Análise nacional do setor financeiro",
    "Base: Silver (dados completos do Brasil)",
    f"Ano de referência: {ANO_ANALISE}",
    separador
])

# Iteração sobre o array para exibição
for linha in mensagens_header:
    print(linha)


ANÁLISES DO SETOR FINANCEIRO - BRASIL

Escopo: Análise nacional do setor financeiro
Base: Silver (dados completos do Brasil)
Ano de referência: 2025


---

## 🔍 ANÁLISES PREPARATÓRIAS

### Contexto
Preparação e limpeza dos dados do setor financeiro para análises mais profundas, incluindo:
- Carregamento e filtragem da base Silver
- Extração e vetorização de dados
- Verificação de integridade e mapeamento de colunas
- Validação de dados e diagnóstico de qualidade

### Atividades
1. Carregamento da base completa do Brasil
2. Filtragem para setor financeiro
3. Criação de arrays NumPy otimizados
4. Mapeamento de colunas e verificação de existência
5. Cálculo de valores nulos e percentuais de ausência
6. Diagnóstico de dados filtrados vs mantidos

---

In [7]:
# CARREGAMENTO E FILTRAGEM - BRASIL

# Formatação de cabeçalho com NumPy
separador = str(np.char.multiply('=', 80))
print("\n" + separador)
print("ETAPA 1: CARREGAMENTO DA BASE SILVER (BRASIL)")
print(separador)

# Carregar base completa --> garantir que a base seja carregada corretamente e que o usuário tenha uma visão clara do conteúdo da base antes de prosseguir com as análises secundárias.
df_brasil = carregar_base_silver()
info_base(df_brasil)

print("\n" + separador)
print("ETAPA 2: FILTRAGEM DO SETOR FINANCEIRO")
print(separador)

# Identificar coluna de segmento --> garantir que a filtragem seja realizada com base na coluna correta, utilizando verificação vetorizada de existência de colunas.
colunas_candidatas = np.array(['segmento_mercado', 'grupo_economico', 'area', 'setor', 'ramo_atividade'])
colunas_df = df_brasil.columns.to_numpy()

# Utilizar np.isin para encontrar intersecção entre colunas desejadas e existentes
# Retorna as colunas candidatas que de fato existem no DataFrame
matches = colunas_candidatas[np.isin(colunas_candidatas, colunas_df)]

# Selecionar a primeira correspondência encontrada (prioridade definida pela ordem do array)
coluna_setor = matches[0] if matches.size > 0 else None

if coluna_setor:
    print(f"✅ Coluna de segmento encontrada: {coluna_setor}")
    
    # Mostrar amostra dos segmentos
    print(f"\nTop 15 segmentos na base:")
    print(df_brasil[coluna_setor].value_counts().head(15))
    
    # Definição de palavras-chave em Array NumPy
    palavras_chave = np.array([
        'banco', 'financ', 'crédito', 'cartão', 'fintech',
        'investimento', 'seguro', 'consórcio', 'empréstimo',
        'pagamento', 'corretora', 'asset', 'capital',
        'previdência', 'capitalização', 'factoring', 'cooperativa'
    ])
    
    # Aplicar filtro utilizando Regex compilado a partir do array NumPy
    # A junção do array é feita para criar o padrão regex 'banco|financ|...'
    padrao_regex = '|'.join(palavras_chave)
    
    # Criação da máscara booleana
    mascara = df_brasil[coluna_setor].str.contains(
        padrao_regex,
        case=False,
        na=False
    )
    
    # Filtragem
    df_financeiro_br = df_brasil[mascara].copy()
    
    # Cálculos estatísticos com NumPy
    qtd_total = df_brasil.shape[0]
    qtd_fin = df_financeiro_br.shape[0]
    percentual = np.round(np.divide(qtd_fin, qtd_total) * 100, 2)
    
    print(f"\n{separador}")
    print("RESULTADO DO FILTRO - SETOR FINANCEIRO BRASIL")
    print(separador)
    print(f"Total de registros Brasil: {qtd_total:,}")
    print(f"Registros setor financeiro: {qtd_fin:,}")
    print(f"Percentual do total: {percentual}%")
    print(separador)

else:
    # Caso nenhuma coluna seja encontrada
    print("\n⚠️ Coluna de segmento não encontrada automaticamente")
    print("Colunas disponíveis:")
    
    # Iteração enumerada sobre o array de colunas
    for i, col in enumerate(colunas_df, 1):
        print(f"  {i}. {col}")
        
    # Fallback: mantém o dataframe original se não filtrar
    df_financeiro_br = df_brasil.copy()


ETAPA 1: CARREGAMENTO DA BASE SILVER (BRASIL)
Carregando de: c:\Users\caroline.coutinho\projeto_mediacao_bancaria\analises\silver\notebooks_silver\..\..\..\data\silver\consumidor_gov_silver_v1.csv
Base carregada com sucesso!
Registros: 2,567,095
Colunas: 30
INFORMACOES DA BASE

Total de registros: 2,567,095
Total de colunas: 30
Periodo: 2024 a 2025
Memoria utilizada: 4203.42 MB

ETAPA 2: FILTRAGEM DO SETOR FINANCEIRO
✅ Coluna de segmento encontrada: area

Top 15 segmentos na base:
area
Serviços Financeiros                        1537096
Demais Serviços                              215452
Telecomunicações                             203578
Transportes                                  123220
Água, Energia, Gás                           110466
Demais Produtos                              109925
Produtos de Telefonia e Informática           83726
Produtos Eletrodomésticos e Eletrônicos       68062
Saúde                                         46797
Turismo/Viagens                         

In [8]:
# 1. PREPARAÇÃO DOS DADOS (VETORIZAÇÃO)

print(f"Iniciando extração de arrays NumPy para {len(df_financeiro_br)} registros...")

# Extração dos dados para arrays NumPy (Ganho de performance)
# Ao converter agora, evitamos o overhead do Pandas nas próximas células
anos_array = df_financeiro_br['ano_abertura'].to_numpy(dtype=np.int16) # Otimiza memória
datas_array = pd.to_datetime(df_financeiro_br['data_abertura'], errors='coerce').to_numpy()

print("✅ Arrays 'anos_array' e 'datas_array' criados com sucesso.")
print(f"   Memória otimizada: Tipo de dado do ano ajustado para {anos_array.dtype}")

Iniciando extração de arrays NumPy para 1537096 registros...
✅ Arrays 'anos_array' e 'datas_array' criados com sucesso.
   Memória otimizada: Tipo de dado do ano ajustado para int16


In [9]:
# 2. ANÁLISE ESTATÍSTICA ANUAL

# 1. Análise de Distribuição por Ano (np.unique é mais rápido que value_counts)
anos_unicos, contagens = np.unique(anos_array, return_counts=True)

# Cálculo vetorizado de percentuais
total_registros = anos_array.size
percentuais = (contagens / total_registros) * 100

# Criação de DataFrame para apresentação
df_resumo_ano = pd.DataFrame({
    'Ano': anos_unicos,
    'Registros': contagens,
    'Percentual': percentuais
}).set_index('Ano')

print("\nDISTRIBUIÇÃO DETALHADA POR ANO:")

# Aplicação de estilo visual 
display(
    df_resumo_ano.style
    .format({'Percentual': '{:.2f}%', 'Registros': '{:,}'})
    .background_gradient(cmap='Blues', subset=['Registros'])
    .bar(subset=['Percentual'], color="#23b016", vmin=0, vmax=100)
)


DISTRIBUIÇÃO DETALHADA POR ANO:


,Registros,Percentual
Ano,,
2024,"54,018",3.51%
2025,"1,483,078",96.49%


In [10]:
# 1. DEFINIÇÃO E VETORIZAÇÃO DO MAPEAMENTO

print("=" * 80)
print("CONFIGURAÇÃO DE MAPEAMENTO - BRASIL (ARQUITETURA NUMPY)")
print("=" * 80)

# Dicionário Base
colunas_map = {
    'empresa': 'nome_fantasia',
    'data_abertura': 'data_abertura',
    'data_resposta': 'data_resposta',
    'data_finalizacao': 'data_finalizacao',
    'problema': 'assunto',
    'grupo_problema': 'grupo_problema',
    'problema_detalhado': 'problema',
    'estado': 'uf',
    'cidade': 'cidade',
    'regiao': 'regiao',
    'status': 'situacao',
    'nota': 'nota_do_consumidor',
    'avaliacao': 'avaliacao_reclamacao',
    'tempo_resposta': 'tempo_resposta',
    'segmento': 'segmento_de_mercado',
    'sexo': 'sexo',
    'faixa_etaria': 'faixa_etaria',
    'respondida': 'respondida',
    'procurou_empresa': 'procurou_empresa',
    'is_agibank': 'is_agibank'
}

# Conversão para Arrays NumPy (Chaves e Valores separados)
# Isso permite operações de busca muito mais rápidas que iterar dicionários
keys_arr = np.array(list(colunas_map.keys()))
cols_target_arr = np.array(list(colunas_map.values()))
cols_existentes_arr = df_financeiro_br.columns.to_numpy()

print(f"✅ Estruturas vetorizadas criadas para {keys_arr.size} campos.")

CONFIGURAÇÃO DE MAPEAMENTO - BRASIL (ARQUITETURA NUMPY)
✅ Estruturas vetorizadas criadas para 20 campos.


In [11]:
# 1. Verificação de Existência (Operação de Conjunto Vetorizada)
# Retorna array booleano [True, True, False...] indicando se a coluna alvo existe no DF

mask_existencia = np.isin(cols_target_arr, cols_existentes_arr)

# 2. Função Otimizada para Cálculo de Nulos
# Definimos uma função que recebe o NOME da coluna e retorna a qtd de nulos

def calc_nulos(col_name):
    if col_name in df_financeiro_br.columns:
        # Usa values (numpy array subjacente) para contar nulos rapidamente
        return pd.isnull(df_financeiro_br[col_name].values).sum()
    return -1 # Código para 'coluna não encontrada'

# Vetorizamos a função para aplicar no array de nomes de colunas

v_calc_nulos = np.vectorize(calc_nulos)

# 3. Execução dos Cálculos

total_linhas = len(df_financeiro_br)
qtd_nulos_arr = v_calc_nulos(cols_target_arr)

# Cálculo de percentuais (onde a coluna existe)
# np.where evita divisão por zero ou cálculos em colunas inexistentes

pct_nulos_arr = np.where(
    mask_existencia, 
    (qtd_nulos_arr / total_linhas) * 100, 
    100.0 # Se não existe, consideramos 100% ausente
)

# 4. Montagem do Relatório Visual
# Usamos np.column_stack para juntar os arrays em uma matriz de dados

status_icones = np.where(mask_existencia, '✅', '❌')

df_relatorio_numpy = pd.DataFrame({
    'Status': status_icones,
    'Chave Interna': keys_arr,
    'Coluna Alvo': cols_target_arr,
    'Qtd Nulos': np.where(qtd_nulos_arr == -1, total_linhas, qtd_nulos_arr),
    '% Ausência': pct_nulos_arr
})

print("\nRELATÓRIO DE INTEGRIDADE (PROCESSAMENTO VETORIZADO):")
display(
    df_relatorio_numpy.style
    .format({'% Ausência': '{:.1f}%', 'Qtd Nulos': '{:,}'})
    .background_gradient(cmap='RdYlGn_r', subset=['% Ausência'])
)


RELATÓRIO DE INTEGRIDADE (PROCESSAMENTO VETORIZADO):


,Status,Chave Interna,Coluna Alvo,Qtd Nulos,% Ausência
0,✅,empresa,nome_fantasia,0,0.0%
1,✅,data_abertura,data_abertura,0,0.0%
2,✅,data_resposta,data_resposta,"174,677",11.4%
3,✅,data_finalizacao,data_finalizacao,0,0.0%
4,✅,problema,assunto,0,0.0%
5,✅,grupo_problema,grupo_problema,0,0.0%
6,✅,problema_detalhado,problema,0,0.0%
7,✅,estado,uf,0,0.0%
8,✅,cidade,cidade,0,0.0%
9,✅,regiao,regiao,0,0.0%


In [12]:
# ESTATÍSTICAS DESCRITIVAS

print("\n" + "=" * 80)
print("ANÁLISE: NOTA DO CONSUMIDOR NO SETOR FINANCEIRO - BRASIL")
print("=" * 80)

# Extração para Array NumPy (Garante performance máxima)
# Mantemos NaN para contagem correta de nulos
arr_notas_bruto = df_financeiro_br[COL_NOTA_BR].to_numpy(dtype=float)

# Filtragem apenas para cálculos (equivalente ao .dropna() mas em array)
# A máscara ~np.isnan cria um filtro booleano instantâneo
mask_validos = ~np.isnan(arr_notas_bruto)
arr_notas_validas = arr_notas_bruto[mask_validos]

# Métricas de Volumetria
total_registros = arr_notas_bruto.size
qtd_validos = arr_notas_validas.size
qtd_nulos = total_registros - qtd_validos
taxa_resposta = (qtd_validos / total_registros) * 100

print(f"\nESTATÍSTICAS GERAIS")
print("-" * 80)
print(f"Total de reclamações: {total_registros:,}")
print(f"Reclamações com nota: {qtd_validos:,}")
print(f"Reclamações sem nota: {qtd_nulos:,}")
print(f"Taxa de avaliação:    {taxa_resposta:.1f}%")

if qtd_validos > 0:
    # Cálculos Estatísticos Vetorizados (Ignorando NaNs)
    media = np.nanmean(arr_notas)
    mediana = np.nanmedian(arr_notas)
    desvio_padrao = np.nanstd(arr_notas)
    minimo = np.nanmin(arr_notas)
    maximo = np.nanmax(arr_notas)
    
    # Criação de DataFrame para exibição elegante
    df_stats = pd.DataFrame({
        'Métrica': ['Média', 'Mediana', 'Desvio Padrão', 'Mínimo', 'Máximo'],
        'Valor': [media, mediana, desvio_padrao, minimo, maximo]
    })
    
    print(f"\nMÉTRICAS DE TENDÊNCIA CENTRAL E DISPERSÃO:")
    display(
        df_stats.style
        .format({'Valor': '{:.2f}'})
        .background_gradient(cmap='Blues', subset=['Valor'])
    )


ANÁLISE: NOTA DO CONSUMIDOR NO SETOR FINANCEIRO - BRASIL


NameError: name 'COL_NOTA_BR' is not defined

---

## ⭐ ANÁLISES DE SATISFAÇÃO E AVALIAÇÃO

### Contexto
Análise das notas de satisfação dos consumidores no setor financeiro, incluindo:
- Estatísticas descritivas (média, mediana, desvio padrão)
- Distribuição das notas (1-5)
- Classificação de satisfação (insatisfeitos, neutros, satisfeitos)
- Cálculo do Net Promoter Score (NPS)

### Perguntas Respondidas
1. Qual é a nota média de satisfação do setor?
2. Como se distribui a satisfação dos consumidores?
3. Qual é o NPS (Net Promoter Score)?
4. Qual percentual está satisfeito, neutro ou insatisfeito?
5. Qual é a taxa de avaliação das reclamações?

---

In [ ]:
# DISTRIBUIÇÃO DAS NOTAS

if qtd_validos > 0:
    print(f"\nDISTRIBUIÇÃO DAS NOTAS")
    print("-" * 80)
    
    # np.unique substitui value_counts() com mais performance
    notas_unicas, contagens = np.unique(arr_notas_validas, return_counts=True)
    
    # Cálculos vetorizados de percentual (operação em array inteiro)
    pct_avaliadas = (contagens / qtd_validos) * 100
    
    # Geração visual das barras sem loop for complexo
    # Iteramos apenas para o print final
    for nota, count, pct in zip(notas_unicas, contagens, pct_avaliadas):
        barra = "█" * int(pct / 2) # Divisão inteira simples
        print(f"Nota {int(nota)}: {count:8,} ({pct:5.1f}%) {barra}")


DISTRIBUIÇÃO DAS NOTAS
--------------------------------------------------------------------------------
Nota 1:  207,357 ( 56.3%) ████████████████████████████
Nota 2:   21,678 (  5.9%) ██
Nota 3:   34,835 (  9.5%) ████
Nota 4:   26,143 (  7.1%) ███
Nota 5:   78,294 ( 21.3%) ██████████


In [ ]:
# CLASSIFICAÇÃO E NPS


if qtd_validos > 0:
    print(f"\nCLASSIFICAÇÃO DE SATISFAÇÃO")
    print("-" * 80)
    
    # Máscaras booleanas NumPy (True/False)
    mask_insatisfeitos = (arr_notas_validas <= 2)
    mask_neutros       = (arr_notas_validas == 3)
    mask_satisfeitos   = (arr_notas_validas >= 4)
    
    # Contagem rápida somando os Trues
    qtd_insatisfeitos = np.sum(mask_insatisfeitos)
    qtd_neutros       = np.sum(mask_neutros)
    qtd_satisfeitos   = np.sum(mask_satisfeitos)
    
    # Cálculo percentual
    pct_insatisfeitos = (qtd_insatisfeitos / qtd_validos) * 100
    pct_neutros       = (qtd_neutros / qtd_validos) * 100
    pct_satisfeitos   = (qtd_satisfeitos / qtd_validos) * 100
    
    print(f"😞 Insatisfeitos (nota 1-2): {qtd_insatisfeitos:,} ({pct_insatisfeitos:.1f}%)")
    print(f"😐 Neutros (nota 3):        {qtd_neutros:,} ({pct_neutros:.1f}%)")
    print(f"😊 Satisfeitos (nota 4-5):  {qtd_satisfeitos:,} ({pct_satisfeitos:.1f}%)")
    
    # --- CÁLCULO DO NPS ---
    print(f"\nNET PROMOTER SCORE (NPS)")
    print("-" * 80)
    
    # Filtros específicos do NPS (Nota 5 vs Notas 1-2)
    qtd_promotores_nps = np.sum(arr_notas_validas == 5)
    qtd_detratores_nps = np.sum(arr_notas_validas <= 2) # Detratores
    
    # Fórmula vetorizada: (Promotores - Detratores) / Total * 100
    nps = ((qtd_promotores_nps - qtd_detratores_nps) / qtd_validos) * 100
    
    print(f"Promotores (nota 5):   {qtd_promotores_nps:,} ({(qtd_promotores_nps/qtd_validos)*100:.1f}%)")
    print(f"Detratores (nota 1-2): {qtd_detratores_nps:,} ({(qtd_detratores_nps/qtd_validos)*100:.1f}%)")
    print(f"NPS: {nps:.1f}")
    
    # Classificação condicional usando np.select (substitui if/elif/else)
    condicoes = [nps >= 75, nps >= 50, nps >= 0]
    opcoes    = ["Excelente", "Muito Bom", "Razoável"]
    
    # Se nenhuma condição for atendida, usa o default 'Crítico'
    # .item() extrai o valor escalar do array de 0 dimensão resultante
    classificacao = np.select(condicoes, opcoes, default="Crítico").item()
    
    print(f"Classificação: {classificacao}")

print("-" * 80)


CLASSIFICAÇÃO DE SATISFAÇÃO
--------------------------------------------------------------------------------
😞 Insatisfeitos (nota 1-2): 229,035 (62.2%)
😐 Neutros (nota 3):        34,835 (9.5%)
😊 Satisfeitos (nota 4-5):  104,437 (28.4%)

NET PROMOTER SCORE (NPS)
--------------------------------------------------------------------------------
Promotores (nota 5):   78,294 (21.3%)
Detratores (nota 1-2): 229,035 (62.2%)
NPS: -40.9
Classificação: Crítico
--------------------------------------------------------------------------------


In [ ]:
print("="*80)
print("DIAGNÓSTICO: VERIFICANDO SE ESTAMOS PERDENDO DADOS")
print("="*80)

# 1. Total de registros
print(f"\n1. TOTAL DE REGISTROS")
print(f"   DataFrame completo: {len(df):,}")

# 2. Verificar filtros aplicados
print(f"\n2. VERIFICANDO FILTROS")
print(f"   Registros df_financeiro_br: {len(df_financeiro_br):,}")
print(f"   Diferença: {len(df) - len(df_financeiro_br):,}")

# 3. Notas por dataset
print(f"\n3. COMPARAÇÃO DE NOTAS")
print(f"   Notas no df original: {df['nota_do_consumidor'].notna().sum():,}")
print(f"   Notas no df_financeiro_br: {df_financeiro_br['nota_do_consumidor'].notna().sum():,}")

# 4. Verificar se a variável COL_NOTA_BR está correta
print(f"\n4. VERIFICANDO VARIÁVEL")
print(f"   COL_NOTA_BR = '{COL_NOTA_BR}'")
print(f"   Existe no df? {COL_NOTA_BR in df.columns}")
print(f"   Existe no df_financeiro_br? {COL_NOTA_BR in df_financeiro_br.columns}")

# 5. Distribuição completa
print(f"\n5. DISTRIBUIÇÃO COMPLETA DE NOTAS (df original)")
print(df['nota_do_consumidor'].value_counts(dropna=False).sort_index())

print(f"\n6. DISTRIBUIÇÃO COMPLETA DE NOTAS (df_financeiro_br)")
print(df_financeiro_br['nota_do_consumidor'].value_counts(dropna=False).sort_index())

DIAGNÓSTICO: VERIFICANDO SE ESTAMOS PERDENDO DADOS

1. TOTAL DE REGISTROS
   DataFrame completo: 2,567,095

2. VERIFICANDO FILTROS
   Registros df_financeiro_br: 1,537,096
   Diferença: 1,029,999

3. COMPARAÇÃO DE NOTAS
   Notas no df original: 765,889
   Notas no df_financeiro_br: 368,307

4. VERIFICANDO VARIÁVEL
   COL_NOTA_BR = 'nota_do_consumidor'
   Existe no df? True
   Existe no df_financeiro_br? True

5. DISTRIBUIÇÃO COMPLETA DE NOTAS (df original)
nota_do_consumidor
1.00     384924
2.00      38690
3.00      64959
4.00      59430
5.00     217886
NaN     1801206
Name: count, dtype: int64

6. DISTRIBUIÇÃO COMPLETA DE NOTAS (df_financeiro_br)
nota_do_consumidor
1.00     207357
2.00      21678
3.00      34835
4.00      26143
5.00      78294
NaN     1168789
Name: count, dtype: int64


In [ ]:
# 1. IDENTIFICAÇÃO DE REGISTROS FILTRADOS

print("="*80)
print("DIAGNÓSTICO DO FILTRO: O QUE FOI REMOVIDO?")
print("="*80)

# Define o DataFrame de origem (df_brasil se existir, senão df)
df_origem = df_brasil if 'df_brasil' in locals() else df

# Tenta recuperar o nome da coluna de setor usada anteriormente
# Se 'coluna_setor' não estiver definida, tentamos adivinhar entre as comuns
if 'coluna_setor' not in locals() or coluna_setor is None:
    possiveis_colunas = ['segmento_mercado', 'segmento_de_mercado', 'grupo_economico', 'area']
    col_encontrada = next((c for c in possiveis_colunas if c in df_origem.columns), None)
    nome_col_setor = col_encontrada
else:
    nome_col_setor = coluna_setor

print(f"ℹ️ Usando coluna de setor: '{nome_col_setor}'")

if nome_col_setor:
    # Extração de arrays para performance (com tratamento de erro se coluna não existir)
    arr_segmentos_total = df_origem[nome_col_setor].to_numpy().astype(str)
    
    # Verifica se 'nome_fantasia' existe, senão usa a primeira coluna de texto disponível
    col_empresa = 'nome_fantasia' if 'nome_fantasia' in df_origem.columns else df_origem.columns[0]
    arr_empresas_total = df_origem[col_empresa].to_numpy().astype(str)
    
    # Verifica coluna de nota
    col_nota = 'nota_do_consumidor' if 'nota_do_consumidor' in df_origem.columns else None
    if col_nota:
        arr_notas_total = df_origem[col_nota].to_numpy(dtype=float)
    else:
        arr_notas_total = np.full(len(df_origem), np.nan) # Array vazio se não tiver nota

    # Recriando a lógica de filtro
    palavras_chave_arr = np.array([
        'banco', 'financ', 'crédito', 'cartão', 'fintech',
        'investimento', 'seguro', 'consórcio', 'empréstimo',
        'pagamento', 'corretora', 'asset', 'capital',
        'previdência', 'capitalização', 'factoring', 'cooperativa'
    ])
    regex_pat = '|'.join(palavras_chave_arr)

    # Máscara de quem É financeiro
    mask_financeiro = df_origem[nome_col_setor].str.contains(regex_pat, case=False, na=False).to_numpy()
    mask_filtrados = ~mask_financeiro # Inverte: Quem NÃO é financeiro

    # Contagens
    qtd_total = mask_filtrados.size
    qtd_filtrados = np.sum(mask_filtrados)
    qtd_mantidos = np.sum(mask_financeiro)

    print(f"\nRESUMO DA OPERAÇÃO:")
    print(f"• Total Original:      {qtd_total:,}")
    print(f"• Registros Mantidos:  {qtd_mantidos:,} (Setor Financeiro)")
    print(f"• Registros Filtrados: {qtd_filtrados:,} (Outros Setores)")
    print(f"• Taxa de Retenção:    {(qtd_mantidos/qtd_total)*100:.1f}%")

else:
    print("❌ Erro: Não foi possível identificar a coluna de segmento no DataFrame original.")

DIAGNÓSTICO DO FILTRO: O QUE FOI REMOVIDO?
ℹ️ Usando coluna de setor: 'area'

RESUMO DA OPERAÇÃO:
• Total Original:      2,567,095
• Registros Mantidos:  1,537,096 (Setor Financeiro)
• Registros Filtrados: 1,029,999 (Outros Setores)
• Taxa de Retenção:    59.9%


In [ ]:
# CARACTERÍSTICAS DOS DESCARTADOS

if 'mask_filtrados' in locals() and np.sum(mask_filtrados) > 0:
    print("\n" + "="*80)
    print("ANÁLISE DOS DADOS DESCARTADOS")
    print("="*80)

    # Função auxiliar para Top N usando NumPy
    def top_n_numpy(array_dados, n=10):
        unicos, contagens = np.unique(array_dados, return_counts=True)
        indices_top = np.argsort(contagens)[-n:][::-1]
        return unicos[indices_top], contagens[indices_top]

    # 1. Análise por Segmento
    segmentos_filtrados = arr_segmentos_total[mask_filtrados]
    top_seg, count_seg = top_n_numpy(segmentos_filtrados, n=10)

    print("\n1. PRINCIPAIS SEGMENTOS DESCARTADOS:")
    df_top_seg = pd.DataFrame({'Segmento': top_seg, 'Qtd': count_seg})
    display(df_top_seg.style.bar(subset=['Qtd'], color="#131a9c"))

    # 2. Análise por Empresa
    empresas_filtradas = arr_empresas_total[mask_filtrados]
    top_emp, count_emp = top_n_numpy(empresas_filtradas, n=10)

    print("\n2. PRINCIPAIS EMPRESAS DESCARTADAS:")
    df_top_emp = pd.DataFrame({'Empresa': top_emp, 'Qtd': count_emp})
    display(df_top_emp.style.bar(subset=['Qtd'], color='#131a9c'))


ANÁLISE DOS DADOS DESCARTADOS

1. PRINCIPAIS SEGMENTOS DESCARTADOS:


,Segmento,Qtd
0,Demais Serviços,215452
1,Telecomunicações,203578
2,Transportes,123220
3,"Água, Energia, Gás",110466
4,Demais Produtos,109925
5,Produtos de Telefonia e Informática,83726
6,Produtos Eletrodomésticos e Eletrônicos,68062
7,Saúde,46797
8,Turismo/Viagens,26236
9,Educação,21615



2. PRINCIPAIS EMPRESAS DESCARTADAS:


,Empresa,Qtd
0,Vivo - Telefônica,73775
1,Tim,44758
2,Claro Celular,44554
3,Facebook / Instagram,43402
4,Google,41904
5,Azul Linhas Aéreas,38865
6,Claro Residencial,34211
7,Mercado Livre,32555
8,Latam Airlines (Tam),30159
9,Samsung,19928


In [ ]:
print("="*80)
print("COMPARAÇÃO: MANTIDOS vs FILTRADOS")
print("="*80)

comparacao = pd.DataFrame({
    'Mantidos': [
        len(df[~df['foi_filtrado']]),
        df[~df['foi_filtrado']]['nota_do_consumidor'].notna().sum(),
        df[~df['foi_filtrado']]['nota_do_consumidor'].mean()
    ],
    'Filtrados': [
        len(df[df['foi_filtrado']]),
        df[df['foi_filtrado']]['nota_do_consumidor'].notna().sum(),
        df[df['foi_filtrado']]['nota_do_consumidor'].mean()
    ]
}, index=['Total Registros', 'Com Nota', 'Nota Média'])

print(comparacao)

# Diferença de nota média
nota_mantidos = df[~df['foi_filtrado']]['nota_do_consumidor'].mean()
nota_filtrados = df[df['foi_filtrado']]['nota_do_consumidor'].mean()
print(f"\n⚠️ Diferença na nota média: {nota_filtrados - nota_mantidos:+.2f}")

COMPARAÇÃO: MANTIDOS vs FILTRADOS
                  Mantidos  Filtrados
Total Registros 1537096.00 1029999.00
Com Nota         368307.00  397582.00
Nota Média            2.31       2.85

⚠️ Diferença na nota média: +0.54


In [ ]:
print("="*80)
print("SETOR FINANCEIRO - ESTATÍSTICAS DE RECLAMAÇÕES E NOTAS")
print("="*80)

# Total de reclamações
total_reclamacoes = len(df_financeiro_br)

# Reclamações com nota
com_nota = df_financeiro_br['nota_do_consumidor'].notna().sum()

# Reclamações sem nota
sem_nota = df_financeiro_br['nota_do_consumidor'].isna().sum()

# Taxa de avaliação
taxa_avaliacao = (com_nota / total_reclamacoes * 100)

print(f"""
RESUMO GERAL:
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Total de Reclamações:           {total_reclamacoes:>12,}  (100.0%)

✅ Reclamações COM Nota:           {com_nota:>12,}  ({com_nota/total_reclamacoes*100:>5.1f}%)
❌ Reclamações SEM Nota:           {sem_nota:>12,}  ({sem_nota/total_reclamacoes*100:>5.1f}%)

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Taxa de Avaliação: {taxa_avaliacao:.1f}%

""")

# Visualização gráfica simples
print("REPRESENTAÇÃO VISUAL:")
print("━" * 80)

barra_com_nota = "█" * int(taxa_avaliacao / 2)
barra_sem_nota = "░" * int((100 - taxa_avaliacao) / 2)

print(f"COM Nota ({taxa_avaliacao:.1f}%): {barra_com_nota}")
print(f"SEM Nota ({100-taxa_avaliacao:.1f}%): {barra_sem_nota}")

print("━" * 80)

# Detalhamento das notas
print("\n" + "="*80)
print("DISTRIBUIÇÃO DAS NOTAS (apenas reclamações avaliadas)")
print("="*80)

distribuicao = df_financeiro_br['nota_do_consumidor'].value_counts().sort_index()

print(f"\n{'Nota':<10} {'Quantidade':>15} {'% do Total':>15} {'% das Avaliadas':>20}")
print("━" * 80)

for nota, qtd in distribuicao.items():
    pct_total = (qtd / total_reclamacoes * 100)
    pct_avaliadas = (qtd / com_nota * 100)
    barra = "█" * int(pct_avaliadas / 2)
    print(f"{int(nota):<10} {qtd:>15,} {pct_total:>14.1f}% {pct_avaliadas:>19.1f}% {barra}")

print("━" * 80)
print(f"{'TOTAL':<10} {com_nota:>15,} {com_nota/total_reclamacoes*100:>14.1f}% {'100.0%':>20}")

SETOR FINANCEIRO - ESTATÍSTICAS DE RECLAMAÇÕES E NOTAS

RESUMO GERAL:
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Total de Reclamações:              1,537,096  (100.0%)

✅ Reclamações COM Nota:                368,307  ( 24.0%)
❌ Reclamações SEM Nota:              1,168,789  ( 76.0%)

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Taxa de Avaliação: 24.0%


REPRESENTAÇÃO VISUAL:
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
COM Nota (24.0%): ███████████
SEM Nota (76.0%): ░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

DISTRIBUIÇÃO DAS NOTAS (apenas reclamações avaliadas)

Nota            Quantidade      % do Total      % das Avaliadas
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
1                  207,357           13.5%                56.3% ████████████████████████████
2                   21,678     

In [ ]:
# Classificação de satisfação, considerando notas 1-2 como insatisfeitos, nota 3 como neutros e notas 4-5 como satisfeitos, para fornecer uma visão geral do nível de satisfação dos consumidores com base nas avaliações fornecidas.

print("="*80)
print("5. ANÁLISE TEMPORAL - SETOR FINANCEIRO 2025")
print("="*80)

# Converter datas para datetime--> garantir que as colunas de data sejam convertidas corretamente para o formato datetime, permitindo a criação de variáveis temporais e a realização de análises baseadas em tempo, como tendências ao longo dos meses, sazonalidade e períodos de maior ou menor atividade.

df_financeiro_br['dt_abertura'] = pd.to_datetime(df_financeiro_br[COL_DATA_BR], errors='coerce')
df_financeiro_br['dt_resposta'] = pd.to_datetime(df_financeiro_br[COL_DATA_RESP_BR], errors='coerce')
df_financeiro_br['dt_finalizacao'] = pd.to_datetime(df_financeiro_br[COL_DATA_FIM_BR], errors='coerce')

# Criar variáveis temporais --> garantir que as variáveis temporais sejam criadas de forma consistente e que forneçam insights valiosos para as análises subsequentes, permitindo a segmentação dos dados por diferentes períodos de tempo e facilitando a identificação de padrões sazonais ou tendências ao longo do tempo.

df_financeiro_br['ano'] = df_financeiro_br['dt_abertura'].dt.year
df_financeiro_br['mes'] = df_financeiro_br['dt_abertura'].dt.month
df_financeiro_br['mes_nome'] = df_financeiro_br['dt_abertura'].dt.strftime('%b')
df_financeiro_br['ano_mes'] = df_financeiro_br['dt_abertura'].dt.to_period('M')
df_financeiro_br['trimestre'] = df_financeiro_br['dt_abertura'].dt.quarter
df_financeiro_br['semestre'] = df_financeiro_br['dt_abertura'].dt.month.apply(lambda x: 1 if x <= 6 else 2)
df_financeiro_br['dia_semana'] = df_financeiro_br['dt_abertura'].dt.dayofweek
df_financeiro_br['dia_semana_nome'] = df_financeiro_br['dt_abertura'].dt.strftime('%A')

# Exibir resumo das variáveis temporais criadas para verificar se estão corretas e fornecer uma visão geral dos dados temporais disponíveis para análise, permitindo a identificação de padrões sazonais, tendências ao longo do tempo e períodos de maior ou menor atividade no setor financeiro.

print("\n✅ Variáveis temporais criadas:")
print("   - ano, mes, mes_nome")
print("   - ano_mes, trimestre, semestre")
print("   - dia_semana, dia_semana_nome")

# Período da análise, verificando a data mínima e máxima de abertura das reclamações para entender o intervalo de tempo coberto pelos dados do setor financeiro, e para garantir que as análises subsequentes sejam realizadas com base em um período de tempo relevante e consistente.

data_min = df_financeiro_br['dt_abertura'].min()
data_max = df_financeiro_br['dt_abertura'].max()

# Exibir período da análise para fornecer contexto sobre o intervalo de tempo coberto pelos dados do setor financeiro, permitindo a identificação de tendências sazonais, períodos de maior ou menor atividade e garantindo que as análises sejam realizadas com base em um período de tempo relevante e consistente.

print(f"\nPERÍODO DA ANÁLISE:")
print(f"   Data inicial: {data_min.strftime('%d/%m/%Y')}")
print(f"   Data final: {data_max.strftime('%d/%m/%Y')}")
print(f"   Total de dias: {(data_max - data_min).days}")
print(f"   Total de meses: {len(df_financeiro_br['ano_mes'].unique())}")

print("\n" + "="*80)

5. ANÁLISE TEMPORAL - SETOR FINANCEIRO 2025

✅ Variáveis temporais criadas:
   - ano, mes, mes_nome
   - ano_mes, trimestre, semestre
   - dia_semana, dia_semana_nome

PERÍODO DA ANÁLISE:
   Data inicial: 01/11/2024
   Data final: 31/12/2025
   Total de dias: 425
   Total de meses: 14



---

## 📅 ANÁLISES TEMPORAIS

### Contexto
Análise de padrões temporais das reclamações do setor financeiro em 2025, incluindo:
- Distribuição ao longo dos meses
- Identificação de sazonalidade
- Análise de cobertura de dados
- Evolução mensal de volume e taxa de avaliação

### Perguntas Respondidas
1. Como as reclamações se distribuem ao longo de 2025?
2. Qual é o padrão mensal de evolução?
3. Há meses completos vs parciais?
4. Até quando os dados foram coletados em Dezembro?
5. Qual é a taxa de avaliação por mês?

---

In [ ]:
print("="*80)
print("5. ANÁLISE TEMPORAL - VISÃO GERAL (2025)")
print("="*80)

# Preparar dados temporais
df_financeiro_br['ano_mes'] = pd.to_datetime(df_financeiro_br[COL_DATA_BR], errors='coerce').dt.to_period('M')

# Evolução mensal
evolucao_mensal = df_financeiro_br.groupby('ano_mes').agg(
    total_reclamacoes=(COL_NOTA_BR, 'count'),
    com_nota=(COL_NOTA_BR, lambda x: x.notna().sum()),
    taxa_avaliacao=(COL_NOTA_BR, lambda x: x.notna().sum() / len(x) * 100 if len(x) > 0 else 0)
).round(2)

print("\n5.1 VOLUME DE RECLAMAÇÕES POR MÊS")
print("-"*80)
print(evolucao_mensal)


5. ANÁLISE TEMPORAL - VISÃO GERAL (2025)

5.1 VOLUME DE RECLAMAÇÕES POR MÊS
--------------------------------------------------------------------------------
         total_reclamacoes  com_nota  taxa_avaliacao
ano_mes                                             
2024-11                102       102           14.09
2024-12               7314      7314           13.72
2025-01              25209     25209           25.40
2025-02              20326     20326           25.04
2025-03              20798     20798           21.70
2025-04              25510     25510           25.38
2025-05              28738     28738           24.86
2025-06              31653     31653           25.58
2025-07              35741     35741           26.09
2025-08              42014     42014           24.23
2025-09              41991     41991           25.55
2025-10              38749     38749           22.37
2025-11              34829     34829           18.17
2025-12              15333     15333           5

In [ ]:
# Estatísticas básicas
print("\n" + "-"*80)
print("RESUMO DO PERÍODO:")
print("-"*80)
print(f"Total de reclamações no ano: {evolucao_mensal['total_reclamacoes'].sum():,.0f}")
print(f"Média mensal: {evolucao_mensal['total_reclamacoes'].mean():,.0f}")
print(f"Mês com mais reclamações: {evolucao_mensal['total_reclamacoes'].idxmax()} ({evolucao_mensal['total_reclamacoes'].max():,.0f})")
print(f"Mês com menos reclamações: {evolucao_mensal['total_reclamacoes'].idxmin()} ({evolucao_mensal['total_reclamacoes'].min():,.0f})")

# Crescimento
crescimento_total = ((evolucao_mensal['total_reclamacoes'].iloc[-1] / evolucao_mensal['total_reclamacoes'].iloc[0] - 1) * 100)
print(f"\nCrescimento total (Jan a Dez): {crescimento_total:+.1f}%")

print("\n" + "="*80)


--------------------------------------------------------------------------------
RESUMO DO PERÍODO:
--------------------------------------------------------------------------------
Total de reclamações no ano: 368,307
Média mensal: 26,308
Mês com mais reclamações: 2025-08 (42,014)
Mês com menos reclamações: 2024-11 (102)

Crescimento total (Jan a Dez): +14932.4%



In [ ]:
# 1. CÁLCULO DE EVOLUÇÃO TEMPORAL

print("="*80)
print("5. ANÁLISE TEMPORAL - VISÃO GERAL (PROCESSAMENTO VETORIZADO)")
print("="*80)

# 1. Extração e Tratamento de Datas (Vetorizado)
# Convertemos para datetime64[M] (precisão mensal) diretamente
arr_datas = pd.to_datetime(df_financeiro_br[COL_DATA_BR], errors='coerce').to_numpy().astype('datetime64[M]')

# 2. Extração de Notas para verificação de Nulos
arr_notas = df_financeiro_br[COL_NOTA_BR].to_numpy(dtype=float)

# 3. Agrupamento e Contagem (Substitui groupby().count())
# np.unique retorna os meses únicos e a contagem total por mês
meses_unicos, total_por_mes = np.unique(arr_datas[~np.isnat(arr_datas)], return_counts=True)

# 4. Cálculo de "Com Nota" e "Sem Nota" por mês
# Como np.unique não faz agregação múltipla direta, usamos um truque de indexação rápida:
# Criamos um array de índices para iterar apenas sobre os meses únicos (muito rápido)
com_nota_por_mes = []
sem_nota_por_mes = []

for mes in meses_unicos:
    # Máscara para o mês atual
    mask_mes = (arr_datas == mes)
    
    # Recorte das notas desse mês
    notas_mes = arr_notas[mask_mes]
    
    # Contagem rápida usando np.isnan
    nulos = np.isnan(notas_mes).sum()
    validos = notas_mes.size - nulos
    
    com_nota_por_mes.append(validos)
    sem_nota_por_mes.append(nulos)

# Conversão para arrays NumPy para cálculos finais
arr_com_nota = np.array(com_nota_por_mes)
arr_sem_nota = np.array(sem_nota_por_mes)

# 5. Cálculo Vetorizado da Taxa
# Evita divisão por zero usando np.where
arr_taxa = np.divide(
    arr_com_nota, 
    total_por_mes, 
    out=np.zeros_like(arr_com_nota, dtype=float), 
    where=total_por_mes!=0
) * 100

print(f"✅ Processamento concluído para {len(meses_unicos)} períodos mensais.")

5. ANÁLISE TEMPORAL - VISÃO GERAL (PROCESSAMENTO VETORIZADO)
✅ Processamento concluído para 14 períodos mensais.


In [ ]:
# 2. FILTRAGEM E VISUALIZAÇÃO (APENAS 2025)

print("\n5.1 VOLUME DE RECLAMAÇÕES POR MÊS (APENAS 2025)")
print("-"*80)

# 1. Criar Máscara Booleana para Filtrar 2024 (NumPy)
# Extrai o ano das datas numpy e compara com 2025
anos_dos_meses = meses_unicos.astype('datetime64[Y]').astype(int) + 1970 
mask_2025 = (anos_dos_meses == 2025)

# 2. Aplicar Máscara em Todos os Arrays (Slicing Vetorizado)
meses_2025 = meses_unicos[mask_2025]
total_2025 = total_por_mes[mask_2025]
com_nota_2025 = arr_com_nota[mask_2025]
sem_nota_2025 = arr_sem_nota[mask_2025]
taxa_2025 = arr_taxa[mask_2025]

# 3. Montagem do DataFrame Final
df_evolucao_2025 = pd.DataFrame({
    'Mês': meses_2025.astype(str), 
    'Total Reclamações': total_2025,
    'Com Nota': com_nota_2025,
    'Sem Nota': sem_nota_2025,
    'Taxa Avaliação': taxa_2025
})

# Configuração de índice
df_evolucao_2025.set_index('Mês', inplace=True)

# 4. Exibição Estilizada
if not df_evolucao_2025.empty:
    display(
        df_evolucao_2025.style
        .format({
            'Total Reclamações': '{:,}', 
            'Com Nota': '{:,}', 
            'Sem Nota': '{:,}', 
            'Taxa Avaliação': '{:.2f}%'
        })
        .background_gradient(cmap='Blues', subset=['Total Reclamações'])
        .bar(subset=['Taxa Avaliação'], color='#5fba7d', vmin=0, vmax=100)
    )
else:
    print("⚠️ Nenhum dado encontrado para o ano de 2025.")


5.1 VOLUME DE RECLAMAÇÕES POR MÊS (APENAS 2025)
--------------------------------------------------------------------------------


,Total Reclamações,Com Nota,Sem Nota,Taxa Avaliação
Mês,,,,
2025-01,"99,242","25,209","74,033",25.40%
2025-02,"81,159","20,326","60,833",25.04%
2025-03,"95,826","20,798","75,028",21.70%
2025-04,"100,523","25,510","75,013",25.38%
2025-05,"115,608","28,738","86,870",24.86%
2025-06,"123,719","31,653","92,066",25.58%
2025-07,"137,001","35,741","101,260",26.09%
2025-08,"173,389","42,014","131,375",24.23%
2025-09,"164,377","41,991","122,386",25.55%


In [ ]:
# ==============================================================================
# 1. INVESTIGAÇÃO TEMPORAL: ONDE OS DADOS PARAM?
# ==============================================================================

print("="*80)
print("DIAGNÓSTICO DE DEZEMBRO: ANÁLISE DIÁRIA")
print("="*80)

# Extração de datas completas (dia a dia)
arr_datas_full = pd.to_datetime(df_financeiro_br[COL_DATA_BR], errors='coerce').to_numpy()

# Filtros para Novembro e Dezembro de 2025
# Usamos datetime64[M] para pegar o mês inteiro de forma rápida
meses_array = arr_datas_full.astype('datetime64[M]')
mask_nov = (meses_array == np.datetime64('2025-11'))
mask_dez = (meses_array == np.datetime64('2025-12'))

# Extração dos dias específicos
dias_nov = arr_datas_full[mask_nov].astype('datetime64[D]')
dias_dez = arr_datas_full[mask_dez].astype('datetime64[D]')

# Contagem diária (Histograma temporal)
dias_unicos_nov, contagem_nov = np.unique(dias_nov, return_counts=True)
dias_unicos_dez, contagem_dez = np.unique(dias_dez, return_counts=True)

print(f"\nCOMPARAÇÃO DE VOLUMETRIA:")
print(f"• Total Novembro: {dias_nov.size:,} registros")
print(f"• Total Dezembro: {dias_dez.size:,} registros (Queda de {(1 - dias_dez.size/dias_nov.size)*100:.1f}%)")

print(f"\nCOBERTURA DE DATAS EM DEZEMBRO:")
if dias_unicos_dez.size > 0:
    data_min_dez = np.min(dias_unicos_dez)
    data_max_dez = np.max(dias_unicos_dez)
    print(f"• Início: {data_min_dez}")
    print(f"• Fim:    {data_max_dez}")
    print(f"• Dias com dados: {dias_unicos_dez.size} de 31")
    
    # Verificar se parou no meio do mês
    ultimo_dia = pd.to_datetime(data_max_dez).day
    if ultimo_dia < 31:
        print(f"⚠️ ALERTA: Os dados de Dezembro vão apenas até o dia {ultimo_dia}!")
    else:
        print(f"ℹ️ Os dados cobrem o mês todo, mas com volume muito baixo.")
else:
    print("❌ Nenhum dado encontrado em Dezembro.")

# Visualização Rápida dos últimos 5 dias com dados
if dias_unicos_dez.size > 0:
    print("\nÚltimos 5 dias com registros em Dezembro:")
    for dia, qtd in zip(dias_unicos_dez[-5:], contagem_dez[-5:]):
        print(f"  {dia}: {qtd:,} reclamações")

DIAGNÓSTICO DE DEZEMBRO: ANÁLISE DIÁRIA

COMPARAÇÃO DE VOLUMETRIA:
• Total Novembro: 191,665 registros
• Total Dezembro: 27,327 registros (Queda de 85.7%)

COBERTURA DE DATAS EM DEZEMBRO:
• Início: 2025-12-01
• Fim:    2025-12-31
• Dias com dados: 31 de 31
ℹ️ Os dados cobrem o mês todo, mas com volume muito baixo.

Últimos 5 dias com registros em Dezembro:
  2025-12-27: 143 reclamações
  2025-12-28: 111 reclamações
  2025-12-29: 92 reclamações
  2025-12-30: 54 reclamações
  2025-12-31: 8 reclamações


---

## 🏢 ANÁLISES EMPRESARIAIS

### Contexto
Análise de performance e características das empresas (instituições financeiras) no setor, incluindo:
- Identificação das principais instituições
- Volume de reclamações por empresa
- Taxa de resolução e satisfação por empresa
- Benchmarking competitivo
- Análise de problemas específicos por instituição

### Perguntas Respondidas
1. Quais são as principais instituições financeiras com mais reclamações?
2. Como se compara a satisfação entre instituições?
3. Qual é a taxa de resolução por empresa?
4. Quais são os principais problemas reportados por instituição?
5. Como o Agibank se posiciona competitivamente?

### Métricas por Empresa
- Volume total de reclamações
- Taxa de avaliação
- Nota média de satisfação
- NPS (Net Promoter Score)
- Tempo médio de resposta
- Taxa de resolução
- Principais segmentos de produtos
- Principais problemas

---

In [ ]:
# Aqui será possível identificar se os dados de Dezembro estão chegando de forma consistente ou se houve uma interrupção abrupta, o que pode indicar um problema na coleta ou processamento dos dados para esse período.

print("DIAGNÓSTICO: O QUE TEM NO DATAFRAME AGORA?")
print("="*80)

# 1. LISTAR TODAS AS COLUNAS --> verificar quais colunas estão presentes no DataFrame 'df_financeiro_br' para entender a estrutura dos dados disponíveis, identificar quais informações podem ser analisadas e garantir que as colunas necessárias para as análises subsequentes estejam presentes e corretamente nomeadas.

print("\nLISTA DE COLUNAS DISPONÍVEIS:")
cols = df_financeiro_br.columns.to_numpy()
# Imprime em grupos de 4 para ficar legível
for i in range(0, len(cols), 4):
    print(f"   {cols[i:i+4]}")

print("-" * 80)

# 2. QUEM CAIU EM CADA CATEGORIA?
# Verificar quais instituições financeiras estão presentes em cada categoria (bancos digitais, bancos tradicionais, outras financeiras) para entender a composição do setor financeiro representado no DataFrame, identificar quais empresas estão mais ativas em termos de reclamações e garantir que as categorias estejam corretamente atribuídas para análises segmentadas.

print("\nEXEMPLOS DE BANCOS POR CATEGORIA (TOP 5 POR VOLUME):")

# Agrupa por categoria e pega os nomes mais frequentes
# Usamos value_counts() para contar as ocorrências de cada empresa dentro de cada categoria, e head(5) para mostrar apenas as 5 mais comuns, proporcionando uma visão rápida das instituições financeiras mais reclamadas em cada categoria.

for categoria in df_financeiro_br['categoria_instituicao'].unique():
    print(f"\n{categoria}:")
    
    # Filtra o dataframe para essa categoria
    mask = df_financeiro_br['categoria_instituicao'] == categoria
    
    # Conta as empresas mais comuns nessa categoria
    top_empresas = df_financeiro_br[mask]['nome_fantasia'].value_counts().head(5)
    
    for empresa, qtd in top_empresas.items():
        print(f"   - {empresa}: {qtd:,} reclamações")

print("-" * 80)

# 3. QUEM FICOU DE FORA? (ANÁLISE DE 'OUTRAS FINANCEIRAS')
# Analisar quais instituições financeiras estão categorizadas como 'Outras Financeiras' para identificar se há bancos importantes ou relevantes que foram classificados incorretamente, e para garantir que as categorias estejam corretamente atribuídas, permitindo análises mais precisas e segmentadas do setor financeiro.

print("\nQUEM SÃO AS 'OUTRAS FINANCEIRAS'? (TOP 10):")
print("Verifique se algum banco grande caiu aqui por engano:")

mask_outros = df_financeiro_br['categoria_instituicao'] == 'Outras Financeiras'
top_outros = df_financeiro_br[mask_outros]['nome_fantasia'].value_counts().head(10)

display(top_outros.to_frame(name='Volume de Reclamações'))

print(f"\nDICA: Se vir um banco importante na lista acima, adicione o nome dele")
print(f"nas listas 'bancos_digitais' ou 'bancos_tradicionais' no bloco anterior.")

DIAGNÓSTICO: O QUE TEM NO DATAFRAME AGORA?

LISTA DE COLUNAS DISPONÍVEIS:
   ['regiao' 'uf' 'cidade' 'sexo']
   ['faixa_etaria' 'ano_abertura' 'mes_abertura' 'data_abertura']
   ['data_resposta' 'data_finalizacao' 'prazo_resposta' 'tempo_resposta']
   ['nome_fantasia' 'segmento_de_mercado' 'area' 'assunto']
   ['grupo_problema' 'problema' 'como_comprou_contratou' 'procurou_empresa']
   ['respondida' 'situacao' 'avaliacao_reclamacao' 'nota_do_consumidor']
   ['data_source' 'file_origin' 'processed_at' 'file_month']
   ['is_agibank' 'quality_score' 'ano_mes_diagnostico' 'ano_mes']
   ['categoria_instituicao' 'score_eficiencia']
--------------------------------------------------------------------------------

EXEMPLOS DE BANCOS POR CATEGORIA (TOP 5 POR VOLUME):

Outras Financeiras:
   - Serasa Experian : 142,507 reclamações
   - Banco Pan : 43,831 reclamações
   - Banco Bmg: 29,994 reclamações
   - Recovery do Brasil Consultoria : 28,027 reclamações
   - Ativos S.A: 24,581 reclamações

Ba

,Volume de Reclamações
nome_fantasia,
Serasa Experian,142507
Banco Pan,43831
Banco Bmg,29994
Recovery do Brasil Consultoria,28027
Ativos S.A,24581
Banco Daycoval,15529
Will Bank,14508
Banco Mercantil do Brasil,12832
Crefisa,12772



DICA: Se vir um banco importante na lista acima, adicione o nome dele
nas listas 'bancos_digitais' ou 'bancos_tradicionais' no bloco anterior.


In [14]:
print("="*80)
print("PREPARAÇÃO DE ARRAYS NUMPY PARA ANÁLISES")
print("="*80)

# Extração de todas as colunas necessárias para arrays NumPy
arr_sexo = df_financeiro_br['sexo'].fillna('Não Informado').to_numpy()
arr_faixa_etaria = df_financeiro_br['faixa_etaria'].fillna('Não Informado').to_numpy()
arr_regiao = df_financeiro_br['regiao'].fillna('Não Informado').to_numpy()
arr_uf = df_financeiro_br['uf'].fillna('Não Informado').to_numpy()
arr_cidade = df_financeiro_br['cidade'].fillna('Não Informado').to_numpy()
arr_instituicao = df_financeiro_br['nome_fantasia'].fillna('Não Informado').to_numpy()
arr_assunto = df_financeiro_br['assunto'].fillna('Não Informado').to_numpy()
arr_avaliacao = df_financeiro_br['avaliacao_reclamacao'].fillna('').to_numpy()
arr_tempo_resposta = pd.to_numeric(df_financeiro_br['tempo_resposta'], errors='coerce').to_numpy()
arr_produto = df_financeiro_br['como_comprou_contratou'].fillna('Não Informado').to_numpy()
arr_nota_completo = df_financeiro_br['nota_do_consumidor'].to_numpy(dtype=float)
arr_procurou_empresa = df_financeiro_br['procurou_empresa'].fillna('Não Informado').to_numpy()

# Verificar se existe coluna de canal
col_canal = None
colunas_canal = ['canal_origem', 'canal_de_origem', 'origem']
for col in colunas_canal:
    if col in df_financeiro_br.columns:
        col_canal = col
        arr_canal = df_financeiro_br[col].fillna('Não Informado').to_numpy()
        break

if col_canal is None:
    arr_canal = np.full(len(df_financeiro_br), 'Não Informado')

print(f"\nArrays criados com sucesso!")
print(f"Total de registros: {len(arr_uf):,}")
print(f"Colunas extraídas:")
print(f"  - sexo, faixa_etaria, regiao, uf, cidade")
print(f"  - instituicao, assunto, avaliacao")
print(f"  - tempo_resposta, produto, nota_completo")
print(f"  - procurou_empresa, canal")
print("="*80)

PREPARAÇÃO DE ARRAYS NUMPY PARA ANÁLISES

Arrays criados com sucesso!
Total de registros: 1,537,096
Colunas extraídas:
  - sexo, faixa_etaria, regiao, uf, cidade
  - instituicao, assunto, avaliacao
  - tempo_resposta, produto, nota_completo
  - procurou_empresa, canal


In [17]:
print("="*80)
print("ANÁLISE DETALHADA: DISTRIBUIÇÃO NO SUDESTE (ES/MG/RJ/SP)")
print("="*80)
print("\nObjetivo: Justificar foco em São Paulo dentro da região Sudeste")
print("-"*80)

# Filtrar apenas estados do Sudeste
estados_sudeste = ['ES', 'MG', 'RJ', 'SP']
mask_sudeste = np.isin(arr_uf, estados_sudeste)

# Contagem por estado do Sudeste
arr_uf_sudeste = arr_uf[mask_sudeste]
ufs_sudeste, contagens_sudeste = np.unique(arr_uf_sudeste, return_counts=True)

# Ordenação decrescente
idx_sud_ord = np.argsort(contagens_sudeste)[::-1]
ufs_sud_ordenadas = ufs_sudeste[idx_sud_ord]
contagens_sud_ordenadas = contagens_sudeste[idx_sud_ord]

# Cálculos percentuais
total_sudeste = np.sum(contagens_sudeste)
pct_dentro_sudeste = (contagens_sud_ordenadas / total_sudeste) * 100
pct_brasil = (contagens_sud_ordenadas / total_registros) * 100

# Métricas de qualidade por estado do Sudeste
notas_sudeste = np.array([
    np.nanmean(arr_nota_completo[arr_uf == uf])
    for uf in ufs_sud_ordenadas
])

taxa_resolucao_sudeste = np.array([
    (np.sum((arr_uf == uf) & (arr_avaliacao == 'Resolvida')) / np.sum(arr_uf == uf)) * 100
    for uf in ufs_sud_ordenadas
])

tempo_medio_sudeste = np.array([
    np.nanmean(arr_tempo_resposta[arr_uf == uf])
    for uf in ufs_sud_ordenadas
])

# DataFrame resultado
df_sudeste = pd.DataFrame({
    'Estado': ufs_sud_ordenadas,
    'Reclamações': contagens_sud_ordenadas,
    '% Sudeste': pct_dentro_sudeste,
    '% Brasil': pct_brasil,
    'Nota Média': notas_sudeste,
    'Taxa Resolução': taxa_resolucao_sudeste,
    'Tempo Resposta (dias)': tempo_medio_sudeste
})

display(
    df_sudeste.style
    .format({
        '% Sudeste': '{:.2f}%',
        '% Brasil': '{:.2f}%',
        'Reclamações': '{:,}',
        'Nota Média': '{:.2f}',
        'Taxa Resolução': '{:.1f}%',
        'Tempo Resposta (dias)': '{:.1f}'
    })
    .background_gradient(cmap='Blues', subset=['Reclamações'])
    .background_gradient(cmap='YlOrBr', subset=['Nota Média'], vmin=1, vmax=5)
    .background_gradient(cmap='Greens', subset=['Taxa Resolução'], vmin=0, vmax=100)
)

# Análise de São Paulo
pct_sp_sudeste = pct_dentro_sudeste[ufs_sud_ordenadas == 'SP'][0]
pct_sp_brasil = pct_brasil[ufs_sud_ordenadas == 'SP'][0]

print(f"\nDESTAQUE SÃO PAULO:")
print(f"  Representa {pct_sp_sudeste:.1f}% do Sudeste")
print(f"  Representa {pct_sp_brasil:.1f}% do Brasil")
print(f"  Volume: {contagens_sud_ordenadas[0]:,} reclamações")
print(f"\nJUSTIFICATIVA: SP concentra mais da metade do Sudeste e é o maior mercado isolado do país")

ANÁLISE DETALHADA: DISTRIBUIÇÃO NO SUDESTE (ES/MG/RJ/SP)

Objetivo: Justificar foco em São Paulo dentro da região Sudeste
--------------------------------------------------------------------------------


,Estado,Reclamações,% Sudeste,% Brasil,Nota Média,Taxa Resolução,Tempo Resposta (dias)
0,SP,"396,999",53.36%,25.83%,2.30,8.2%,5.9
1,MG,"174,130",23.41%,11.33%,2.23,7.5%,6.2
2,RJ,"145,345",19.54%,9.46%,2.30,7.8%,6.0
3,ES,"27,499",3.70%,1.79%,2.33,8.7%,6.0



DESTAQUE SÃO PAULO:
  Representa 53.4% do Sudeste
  Representa 25.8% do Brasil
  Volume: 396,999 reclamações

JUSTIFICATIVA: SP concentra mais da metade do Sudeste e é o maior mercado isolado do país


In [ ]:
print("="*80)
print("FAIXA ETÁRIA QUE MAIS CONCENTRA RECLAMAÇÕES")
print("="*80)
print("\nObjetivo: Identificar perfil etário (jovem digital vs sênior)")
print("-"*80)

# Contagem por faixa etária
faixas_unicas, contagens_faixa = np.unique(arr_faixa_etaria, return_counts=True)

# Ordenação decrescente
idx_ordenado = np.argsort(contagens_faixa)[::-1]
faixas_ordenadas = faixas_unicas[idx_ordenado]
contagens_ordenadas = contagens_faixa[idx_ordenado]

# Percentuais
pct_faixas = (contagens_ordenadas / total_registros) * 100

# Nota média por faixa
notas_por_faixa = np.array([
    np.nanmean(arr_nota_completo[arr_faixa_etaria == faixa])
    for faixa in faixas_ordenadas
])

# Taxa de resolução por faixa
taxa_resolucao_faixa = np.array([
    (np.sum((arr_faixa_etaria == faixa) & (arr_avaliacao == 'Resolvida')) / np.sum(arr_faixa_etaria == faixa)) * 100
    for faixa in faixas_ordenadas
])

# Tempo médio por faixa
tempo_medio_faixa = np.array([
    np.nanmean(arr_tempo_resposta[arr_faixa_etaria == faixa])
    for faixa in faixas_ordenadas
])

# DataFrame resultado
df_faixa_etaria = pd.DataFrame({
    'Faixa Etária': faixas_ordenadas,
    'Reclamações': contagens_ordenadas,
    '% Total': pct_faixas,
    'Nota Média': notas_por_faixa,
    'Taxa Resolução': taxa_resolucao_faixa,
    'Tempo Resposta (dias)': tempo_medio_faixa
})

display(
    df_faixa_etaria.style
    .format({
        '% Total': '{:.2f}%',
        'Reclamações': '{:,}',
        'Nota Média': '{:.2f}',
        'Taxa Resolução': '{:.1f}%',
        'Tempo Resposta (dias)': '{:.1f}'
    })
    .background_gradient(cmap='YlOrRd', subset=['Reclamações'])
    .background_gradient(cmap='RdYlGn', subset=['Nota Média'], vmin=1, vmax=5)
    .background_gradient(cmap='RdYlGn', subset=['Taxa Resolução'], vmin=0, vmax=100)
)

# Análise da faixa predominante
print(f"\nFaixa etária predominante: {faixas_ordenadas[0]}")
print(f"Volume: {contagens_ordenadas[0]:,} ({pct_faixas[0]:.1f}%)")

# Faixa mais insatisfeita
idx_menor_nota = np.nanargmin(notas_por_faixa)
print(f"\nFaixa mais insatisfeita: {faixas_ordenadas[idx_menor_nota]}")
print(f"Nota média: {notas_por_faixa[idx_menor_nota]:.2f}")

# Concentração top 3
top3_concentracao = np.sum(pct_faixas[:3])
print(f"\nConcentração top 3 faixas: {top3_concentracao:.1f}%")

FAIXA ETÁRIA QUE MAIS CONCENTRA RECLAMAÇÕES

Objetivo: Identificar perfil etário (jovem digital vs sênior)
--------------------------------------------------------------------------------


,Faixa Etária,Reclamações,% Total,Nota Média,Taxa Resolução,Tempo Resposta (dias)
0,entre 31 a 40 anos,"458,692",29.84%,2.31,8.7%,5.9
1,entre 21 a 30 anos,"361,631",23.53%,2.35,7.0%,5.5
2,entre 41 a 50 anos,"333,464",21.69%,2.28,8.9%,6.1
3,entre 51 a 60 anos,"171,244",11.14%,2.34,8.4%,6.4
4,entre 61 a 70 anos,"125,280",8.15%,2.27,6.2%,6.8
5,mais de 70 anos,"54,489",3.54%,2.15,4.9%,7.0
6,até 20 anos,"32,296",2.10%,2.51,6.0%,4.6



Faixa etária predominante: entre 31 a 40 anos
Volume: 458,692 (29.8%)

Faixa mais insatisfeita: mais de 70 anos
Nota média: 2.15

Concentração top 3 faixas: 75.1%


In [ ]:
print("="*80)
print("VOLUME DE RECLAMAÇÕES POR REGIÃO")
print("="*80)
print("\nObjetivo: Identificar concentração geográfica (funil para SP)")
print("-"*80)

# Contagem por região
regioes_unicas, contagens_regiao = np.unique(arr_regiao, return_counts=True)

# Ordenação decrescente
idx_regiao_ord = np.argsort(contagens_regiao)[::-1]
regioes_ordenadas = regioes_unicas[idx_regiao_ord]
contagens_regiao_ord = contagens_regiao[idx_regiao_ord]

# Percentuais
pct_regioes = (contagens_regiao_ord / total_registros) * 100

# Nota média por região
notas_por_regiao = np.array([
    np.nanmean(arr_nota_completo[arr_regiao == regiao])
    for regiao in regioes_ordenadas
])

# Taxa de resolução por região
taxa_resolucao_regiao = np.array([
    (np.sum((arr_regiao == regiao) & (arr_avaliacao == 'Resolvida')) / np.sum(arr_regiao == regiao)) * 100
    for regiao in regioes_ordenadas
])

# DataFrame resultado
df_regiao = pd.DataFrame({
    'Região': regioes_ordenadas,
    'Reclamações': contagens_regiao_ord,
    '% Brasil': pct_regioes,
    'Nota Média': notas_por_regiao,
    'Taxa Resolução': taxa_resolucao_regiao
})

display(
    df_regiao.style
    .format({
        '% Brasil': '{:.2f}%',
        'Reclamações': '{:,}',
        'Nota Média': '{:.2f}',
        'Taxa Resolução': '{:.1f}%'
    })
    .background_gradient(cmap='Oranges', subset=['Reclamações'])
    .background_gradient(cmap='RdYlGn', subset=['Nota Média'], vmin=1, vmax=5)
    .background_gradient(cmap='RdYlGn', subset=['Taxa Resolução'], vmin=0, vmax=100)
)

# Análise da região predominante
print(f"\nRegião predominante: {regioes_ordenadas[0]}")
print(f"Volume: {contagens_regiao_ord[0]:,} ({pct_regioes[0]:.1f}%)")
print(f"Nota média: {notas_por_regiao[0]:.2f}")

# Concentração Sudeste
if 'Sudeste' in regioes_ordenadas:
    idx_sudeste = np.where(regioes_ordenadas == 'Sudeste')[0][0]
    print(f"\nSudeste representa {pct_regioes[idx_sudeste]:.1f}% das reclamações nacionais")

VOLUME DE RECLAMAÇÕES POR REGIÃO

Objetivo: Identificar concentração geográfica (funil para SP)
--------------------------------------------------------------------------------


,Região,Reclamações,% Brasil,Nota Média,Taxa Resolução
0,SE,"743,973",48.40%,2.28,8.0%
1,NE,"297,414",19.35%,2.32,7.9%
2,S,"241,997",15.74%,2.31,7.8%
3,CO,"159,433",10.37%,2.36,8.5%
4,N,"94,279",6.13%,2.45,7.5%



Região predominante: SE
Volume: 743,973 (48.4%)
Nota média: 2.28


In [ ]:
print("="*80)
print("DISTRIBUIÇÃO POR ESTADO (TOP 10)")
print("="*80)
print("\nObjetivo: Identificar estados com maior volume (destaque para SP)")
print("-"*80)

# Contagem por UF
ufs_unicas, contagens_uf = np.unique(arr_uf, return_counts=True)

# Ordenação decrescente e seleção top 10
idx_uf_ord = np.argsort(contagens_uf)[::-1]
top10_ufs = ufs_unicas[idx_uf_ord][:10]
top10_contagens = contagens_uf[idx_uf_ord][:10]

# Percentuais
pct_ufs = (top10_contagens / total_registros) * 100

# Nota média por UF
notas_por_uf = np.array([
    np.nanmean(arr_nota_completo[arr_uf == uf])
    for uf in top10_ufs
])

# Taxa de resolução por UF
taxa_resolucao_uf = np.array([
    (np.sum((arr_uf == uf) & (arr_avaliacao == 'Resolvida')) / np.sum(arr_uf == uf)) * 100
    for uf in top10_ufs
])

# DataFrame resultado
df_uf_top10 = pd.DataFrame({
    'UF': top10_ufs,
    'Reclamações': top10_contagens,
    '% Brasil': pct_ufs,
    'Nota Média': notas_por_uf,
    'Taxa Resolução': taxa_resolucao_uf
})

display(
    df_uf_top10.style
    .format({
        '% Brasil': '{:.2f}%',
        'Reclamações': '{:,}',
        'Nota Média': '{:.2f}',
        'Taxa Resolução': '{:.1f}%'
    })
    .background_gradient(cmap='Reds', subset=['Reclamações'])
    .background_gradient(cmap='RdYlGn', subset=['Nota Média'], vmin=1, vmax=5)
)

# Análise de São Paulo
mask_sp = (arr_uf == 'SP')
qtd_sp = np.sum(mask_sp)
pct_sp = (qtd_sp / total_registros) * 100
nota_sp = np.nanmean(arr_nota_completo[mask_sp])

print(f"\nDESTAQUE SÃO PAULO:")
print(f"  Reclamações: {qtd_sp:,} ({pct_sp:.1f}% do Brasil)")
print(f"  Nota média: {nota_sp:.2f}")
print(f"  Posição no ranking: {np.where(top10_ufs == 'SP')[0][0] + 1}º lugar")

DISTRIBUIÇÃO POR ESTADO (TOP 10)

Objetivo: Identificar estados com maior volume (destaque para SP)
--------------------------------------------------------------------------------


,UF,Reclamações,% Brasil,Nota Média,Taxa Resolução
0,SP,"396,999",25.83%,2.30,8.2%
1,MG,"174,130",11.33%,2.23,7.5%
2,RJ,"145,345",9.46%,2.30,7.8%
3,PR,"96,572",6.28%,2.31,8.6%
4,BA,"88,772",5.78%,2.18,7.3%
5,RS,"77,097",5.02%,2.25,7.0%
6,SC,"68,328",4.45%,2.36,7.5%
7,GO,"60,046",3.91%,2.33,7.8%
8,PE,"46,178",3.00%,2.42,8.7%
9,DF,"45,627",2.97%,2.37,10.8%



DESTAQUE SÃO PAULO:
  Reclamações: 396,999 (25.8% do Brasil)
  Nota média: 2.30
  Posição no ranking: 1º lugar


In [ ]:
print("="*80)
print("TOP 10 INSTITUIÇÕES FINANCEIRAS MAIS RECLAMADAS")
print("="*80)
print("\nObjetivo: Identificar principais players geradores de insatisfação")
print("-"*80)

# Contagem por instituição
instituicoes_unicas, contagens_inst = np.unique(arr_instituicao, return_counts=True)

# Ordenação decrescente e seleção top 10
idx_inst_ord = np.argsort(contagens_inst)[::-1]
top10_inst = instituicoes_unicas[idx_inst_ord][:10]
top10_contagens_inst = contagens_inst[idx_inst_ord][:10]

# Percentuais
pct_inst = (top10_contagens_inst / total_registros) * 100

# Nota média por instituição
notas_por_inst = np.array([
    np.nanmean(arr_nota_completo[arr_instituicao == inst])
    for inst in top10_inst
])

# Taxa de resolução por instituição
taxa_resolucao_inst = np.array([
    (np.sum((arr_instituicao == inst) & (arr_avaliacao == 'Resolvida')) / np.sum(arr_instituicao == inst)) * 100
    for inst in top10_inst
])

# Tempo médio de resposta por instituição
tempo_medio_inst = np.array([
    np.nanmean(arr_tempo_resposta[arr_instituicao == inst])
    for inst in top10_inst
])

# DataFrame resultado
df_top10_inst = pd.DataFrame({
    'Instituição': top10_inst,
    'Reclamações': top10_contagens_inst,
    '% Total': pct_inst,
    'Nota Média': notas_por_inst,
    'Taxa Resolução': taxa_resolucao_inst,
    'Tempo Resposta (dias)': tempo_medio_inst
})

display(
    df_top10_inst.style
    .format({
        '% Total': '{:.2f}%',
        'Reclamações': '{:,}',
        'Nota Média': '{:.2f}',
        'Taxa Resolução': '{:.1f}%',
        'Tempo Resposta (dias)': '{:.1f}'
    })
    .background_gradient(cmap='Reds', subset=['Reclamações'])
    .background_gradient(cmap='RdYlGn', subset=['Nota Média'], vmin=1, vmax=5)
    .background_gradient(cmap='RdYlGn', subset=['Taxa Resolução'], vmin=0, vmax=100)
)

# Análise da instituição líder
print(f"\nInstituição mais reclamada: {top10_inst[0]}")
print(f"Volume: {top10_contagens_inst[0]:,} ({pct_inst[0]:.1f}%)")
print(f"Nota média: {notas_por_inst[0]:.2f}")

# Concentração top 5
top5_concentracao = np.sum(pct_inst[:5])
print(f"\nTop 5 instituições concentram: {top5_concentracao:.1f}% das reclamações")

QUESTÃO 4: TOP 10 INSTITUIÇÕES FINANCEIRAS MAIS RECLAMADAS

Objetivo: Identificar principais players geradores de insatisfação
--------------------------------------------------------------------------------


,Instituição,Reclamações,% Total,Nota Média,Taxa Resolução,Tempo Resposta (dias)
0,Nubank,"150,012",9.76%,1.85,3.4%,3.5
1,Serasa Experian,"142,507",9.27%,2.64,10.0%,3.6
2,Banco Santander,"84,948",5.53%,2.10,7.0%,5.8
3,Banco do Brasil,"83,833",5.45%,2.48,7.7%,4.7
4,Banco Bradesco,"77,519",5.04%,2.00,5.1%,8.8
5,Caixa Econômica Federal,"69,196",4.50%,2.65,10.1%,6.8
6,Banco Itaú Unibanco,"52,979",3.45%,2.07,6.8%,6.8
7,Banco Pan,"43,831",2.85%,1.92,3.6%,6.0
8,Mercado Pago,"36,344",2.36%,2.61,12.6%,8.0
9,Cartões Itaú,"31,664",2.06%,2.29,8.7%,8.3



Instituição mais reclamada: Nubank
Volume: 150,012 (9.8%)
Nota média: 1.85

Top 5 instituições concentram: 35.1% das reclamações


In [18]:
print("="*80)
print("TOP 10 CIDADES NÃO-CAPITAIS COM MAIS RECLAMAÇÕES")
print("="*80)
print("\nObjetivo: Verificar se Campinas aparece no radar nacional")
print("-"*80)

# Lista de capitais brasileiras
capitais = [
    'Rio Branco', 'Maceió', 'Macapá', 'Manaus', 'Salvador', 'Fortaleza',
    'Brasília', 'Vitória', 'Goiânia', 'São Luís', 'Cuiabá', 'Campo Grande',
    'Belo Horizonte', 'Belém', 'João Pessoa', 'Curitiba', 'Recife', 'Teresina',
    'Rio de Janeiro', 'Natal', 'Porto Alegre', 'Porto Velho', 'Boa Vista',
    'Florianópolis', 'São Paulo', 'Aracaju', 'Palmas'
]

# Criar máscara para cidades não-capitais
arr_cidade = df_financeiro_br['cidade'].fillna('Não Informado').to_numpy()
mask_nao_capital = ~np.isin(arr_cidade, capitais)

# Filtrar apenas cidades não-capitais
arr_cidade_nao_capital = arr_cidade[mask_nao_capital]

# Contagem por cidade
cidades_unicas, contagens_cidade = np.unique(arr_cidade_nao_capital, return_counts=True)

# Ordenação e seleção top 10
idx_cidade_ord = np.argsort(contagens_cidade)[::-1]
top10_cidades = cidades_unicas[idx_cidade_ord][:10]
top10_contagens_cidade = contagens_cidade[idx_cidade_ord][:10]

# Percentuais
pct_cidades = (top10_contagens_cidade / total_registros) * 100

# UF de cada cidade (pegar a UF mais frequente para cada cidade)
uf_por_cidade = []
for cidade in top10_cidades:
    mask_cidade = (arr_cidade == cidade)
    ufs_cidade = arr_uf[mask_cidade]
    uf_predominante = np.unique(ufs_cidade, return_counts=True)[0][0]
    uf_por_cidade.append(uf_predominante)

uf_por_cidade = np.array(uf_por_cidade)

# Nota média por cidade
notas_por_cidade = np.array([
    np.nanmean(arr_nota_completo[arr_cidade == cidade])
    for cidade in top10_cidades
])

# Taxa de resolução por cidade
taxa_resolucao_cidade = np.array([
    (np.sum((arr_cidade == cidade) & (arr_avaliacao == 'Resolvida')) / np.sum(arr_cidade == cidade)) * 100
    for cidade in top10_cidades
])

# DataFrame resultado
df_top10_cidades = pd.DataFrame({
    'Cidade': top10_cidades,
    'UF': uf_por_cidade,
    'Reclamações': top10_contagens_cidade,
    '% Brasil': pct_cidades,
    'Nota Média': notas_por_cidade,
    'Taxa Resolução': taxa_resolucao_cidade
})

display(
    df_top10_cidades.style
    .format({
        '% Brasil': '{:.2f}%',
        'Reclamações': '{:,}',
        'Nota Média': '{:.2f}',
        'Taxa Resolução': '{:.1f}%'
    })
    .background_gradient(cmap='Blues', subset=['Reclamações'])
    .background_gradient(cmap='YlOrBr', subset=['Nota Média'], vmin=1, vmax=5)
    .background_gradient(cmap='Greens', subset=['Taxa Resolução'], vmin=0, vmax=100)
)

# Verificar posição de Campinas
if 'Campinas' in top10_cidades:
    idx_campinas = np.where(top10_cidades == 'Campinas')[0][0]
    posicao_campinas = idx_campinas + 1
    qtd_campinas = top10_contagens_cidade[idx_campinas]
    pct_campinas = pct_cidades[idx_campinas]
    
    print(f"\nCAMPINAS IDENTIFICADA:")
    print(f"  Posição no ranking nacional (não-capitais): {posicao_campinas}º lugar")
    print(f"  Volume: {qtd_campinas:,} reclamações ({pct_campinas:.2f}% do Brasil)")
    print(f"  Nota média: {notas_por_cidade[idx_campinas]:.2f}")
    print(f"\nJUSTIFICATIVA: Campinas aparece no radar nacional, validando análise municipal específica")
else:
    # Buscar Campinas fora do top 10
    mask_campinas = (arr_cidade == 'Campinas')
    if np.sum(mask_campinas) > 0:
        qtd_campinas = np.sum(mask_campinas)
        pct_campinas = (qtd_campinas / total_registros) * 100
        
        # Encontrar posição exata
        todas_contagens = np.sort(contagens_cidade)[::-1]
        posicao_campinas = np.where(todas_contagens == qtd_campinas)[0][0] + 1
        
        print(f"\nCAMPINAS FORA DO TOP 10:")
        print(f"  Posição: {posicao_campinas}º lugar")
        print(f"  Volume: {qtd_campinas:,} reclamações ({pct_campinas:.2f}%)")
    else:
        print(f"\nCAMPINAS: Não encontrada na base de dados")

TOP 10 CIDADES NÃO-CAPITAIS COM MAIS RECLAMAÇÕES

Objetivo: Verificar se Campinas aparece no radar nacional
--------------------------------------------------------------------------------


,Cidade,UF,Reclamações,% Brasil,Nota Média,Taxa Resolução
0,Guarulhos,SP,"13,170",0.86%,2.34,8.6%
1,Campinas,AC,"10,941",0.71%,2.22,8.2%
2,Uberlândia,MG,"9,167",0.60%,2.21,7.3%
3,Sorocaba,SP,"8,790",0.57%,2.31,8.2%
4,Contagem,MG,"8,041",0.52%,2.15,7.6%
5,Osasco,SP,"7,968",0.52%,2.31,8.5%
6,Joinville,SC,"7,815",0.51%,2.37,8.6%
7,São Bernardo do Campo,SP,"7,420",0.48%,2.25,8.0%
8,Ribeirão Preto,SP,"7,414",0.48%,2.17,7.1%
9,Nova Iguaçu,RJ,"7,365",0.48%,2.30,7.7%



CAMPINAS IDENTIFICADA:
  Posição no ranking nacional (não-capitais): 2º lugar
  Volume: 10,941 reclamações (0.71% do Brasil)
  Nota média: 2.22

JUSTIFICATIVA: Campinas aparece no radar nacional, validando análise municipal específica


In [19]:
print("="*80)
print("TOP 10 CIDADES NÃO-CAPITAIS COM MAIS RECLAMAÇÕES")
print("="*80)
print("\nObjetivo: Verificar se Campinas aparece no radar nacional")
print("-"*80)

# Lista de capitais brasileiras
capitais = [
    'Rio Branco', 'Maceió', 'Macapá', 'Manaus', 'Salvador', 'Fortaleza',
    'Brasília', 'Vitória', 'Goiânia', 'São Luís', 'Cuiabá', 'Campo Grande',
    'Belo Horizonte', 'Belém', 'João Pessoa', 'Curitiba', 'Recife', 'Teresina',
    'Rio de Janeiro', 'Natal', 'Porto Alegre', 'Porto Velho', 'Boa Vista',
    'Florianópolis', 'São Paulo', 'Aracaju', 'Palmas'
]

# Criar máscara para cidades não-capitais
mask_nao_capital = ~np.isin(arr_cidade, capitais)

# Filtrar arrays mantendo os índices originais
indices_nao_capital = np.where(mask_nao_capital)[0]

# Extrair dados filtrados
arr_cidade_filtrado = arr_cidade[mask_nao_capital]
arr_uf_filtrado = arr_uf[mask_nao_capital]
arr_nota_filtrado = arr_nota_completo[mask_nao_capital]
arr_avaliacao_filtrado = arr_avaliacao[mask_nao_capital]

# Contagem por cidade
cidades_unicas, idx_inverse = np.unique(arr_cidade_filtrado, return_inverse=True)
contagens_cidade = np.bincount(idx_inverse)

# Ordenação e seleção top 10
idx_cidade_ord = np.argsort(contagens_cidade)[::-1]
top10_cidades = cidades_unicas[idx_cidade_ord][:10]
top10_contagens_cidade = contagens_cidade[idx_cidade_ord][:10]

# Percentuais
pct_cidades = (top10_contagens_cidade / total_registros) * 100

# Para cada cidade do top 10, pegar a UF correta
uf_por_cidade = []
notas_por_cidade = []
taxa_resolucao_cidade = []

for cidade in top10_cidades:
    # Máscara para esta cidade específica (usando arrays filtrados)
    mask_cidade = (arr_cidade_filtrado == cidade)
    
    # UF mais frequente para esta cidade
    ufs_da_cidade = arr_uf_filtrado[mask_cidade]
    uf_predominante, contagens_uf = np.unique(ufs_da_cidade, return_counts=True)
    uf_correta = uf_predominante[np.argmax(contagens_uf)]
    uf_por_cidade.append(uf_correta)
    
    # Nota média
    notas_da_cidade = arr_nota_filtrado[mask_cidade]
    media = np.nanmean(notas_da_cidade)
    notas_por_cidade.append(media)
    
    # Taxa de resolução
    avaliacoes_da_cidade = arr_avaliacao_filtrado[mask_cidade]
    resolvidas = np.sum(avaliacoes_da_cidade == 'Resolvida')
    taxa = (resolvidas / np.sum(mask_cidade)) * 100 if np.sum(mask_cidade) > 0 else 0
    taxa_resolucao_cidade.append(taxa)

# Converter para arrays
uf_por_cidade = np.array(uf_por_cidade)
notas_por_cidade = np.array(notas_por_cidade)
taxa_resolucao_cidade = np.array(taxa_resolucao_cidade)

# DataFrame resultado
df_top10_cidades = pd.DataFrame({
    'Cidade': top10_cidades,
    'UF': uf_por_cidade,
    'Reclamações': top10_contagens_cidade,
    '% Brasil': pct_cidades,
    'Nota Média': notas_por_cidade,
    'Taxa Resolução': taxa_resolucao_cidade
})

display(
    df_top10_cidades.style
    .format({
        '% Brasil': '{:.2f}%',
        'Reclamações': '{:,}',
        'Nota Média': '{:.2f}',
        'Taxa Resolução': '{:.1f}%'
    })
    .background_gradient(cmap='Blues', subset=['Reclamações'])
    .background_gradient(cmap='YlOrBr', subset=['Nota Média'], vmin=1, vmax=5)
    .background_gradient(cmap='Greens', subset=['Taxa Resolução'], vmin=0, vmax=100)
)

# Verificar posição de Campinas
if 'Campinas' in top10_cidades:
    idx_campinas = np.where(top10_cidades == 'Campinas')[0][0]
    posicao_campinas = idx_campinas + 1
    qtd_campinas = top10_contagens_cidade[idx_campinas]
    pct_campinas = pct_cidades[idx_campinas]
    uf_campinas = uf_por_cidade[idx_campinas]
    
    print(f"\nCAMPINAS IDENTIFICADA:")
    print(f"  UF: {uf_campinas}")
    print(f"  Posição no ranking nacional (não-capitais): {posicao_campinas}º lugar")
    print(f"  Volume: {qtd_campinas:,} reclamações ({pct_campinas:.2f}% do Brasil)")
    print(f"  Nota média: {notas_por_cidade[idx_campinas]:.2f}")
    print(f"\nJUSTIFICATIVA: Campinas aparece no radar nacional,")
    print(f"               validando análise municipal específica")
else:
    # Buscar Campinas fora do top 10
    mask_campinas_total = (arr_cidade == 'Campinas')
    if np.sum(mask_campinas_total) > 0:
        qtd_campinas = np.sum(mask_campinas_total)
        pct_campinas = (qtd_campinas / total_registros) * 100
        
        # UF de Campinas
        uf_campinas_arr = arr_uf[mask_campinas_total]
        uf_campinas = np.unique(uf_campinas_arr, return_counts=True)[0][0]
        
        # Encontrar posição exata
        todas_contagens = np.sort(contagens_cidade)[::-1]
        posicao_campinas = np.searchsorted(-todas_contagens, -qtd_campinas) + 1
        
        print(f"\nCAMPINAS FORA DO TOP 10:")
        print(f"  UF: {uf_campinas}")
        print(f"  Posição: {posicao_campinas}º lugar")
        print(f"  Volume: {qtd_campinas:,} reclamações ({pct_campinas:.2f}%)")
    else:
        print(f"\nCAMPINAS: Não encontrada na base de dados")

TOP 10 CIDADES NÃO-CAPITAIS COM MAIS RECLAMAÇÕES

Objetivo: Verificar se Campinas aparece no radar nacional
--------------------------------------------------------------------------------


,Cidade,UF,Reclamações,% Brasil,Nota Média,Taxa Resolução
0,Guarulhos,SP,"13,170",0.86%,2.34,8.6%
1,Campinas,SP,"10,941",0.71%,2.22,8.2%
2,Uberlândia,MG,"9,167",0.60%,2.21,7.3%
3,Sorocaba,SP,"8,790",0.57%,2.31,8.2%
4,Contagem,MG,"8,041",0.52%,2.15,7.6%
5,Osasco,SP,"7,968",0.52%,2.31,8.5%
6,Joinville,SC,"7,815",0.51%,2.37,8.6%
7,São Bernardo do Campo,SP,"7,420",0.48%,2.25,8.0%
8,Ribeirão Preto,SP,"7,414",0.48%,2.17,7.1%
9,Nova Iguaçu,RJ,"7,365",0.48%,2.30,7.7%



CAMPINAS IDENTIFICADA:
  UF: SP
  Posição no ranking nacional (não-capitais): 2º lugar
  Volume: 10,941 reclamações (0.71% do Brasil)
  Nota média: 2.22

JUSTIFICATIVA: Campinas aparece no radar nacional,
               validando análise municipal específica


In [20]:
print("="*80)
print("TOP 10 ASSUNTOS (TIPOS DE PROBLEMA) MAIS RECLAMADOS")
print("="*80)
print("\nObjetivo: Identificar as principais dores do consumidor brasileiro")
print("-"*80)

# Contagem por assunto
assuntos_unicos, contagens_assunto = np.unique(arr_assunto, return_counts=True)

# Ordenação e seleção top 10
idx_assunto_ord = np.argsort(contagens_assunto)[::-1]
top10_assuntos = assuntos_unicos[idx_assunto_ord][:10]
top10_contagens_assunto = contagens_assunto[idx_assunto_ord][:10]

# Percentuais
pct_assuntos = (top10_contagens_assunto / total_registros) * 100

# Nota média por assunto
notas_por_assunto = np.array([
    np.nanmean(arr_nota_completo[arr_assunto == assunto])
    for assunto in top10_assuntos
])

# Taxa de resolução por assunto
taxa_resolucao_assunto = np.array([
    (np.sum((arr_assunto == assunto) & (arr_avaliacao == 'Resolvida')) / np.sum(arr_assunto == assunto)) * 100
    for assunto in top10_assuntos
])

# Tempo médio por assunto
tempo_medio_assunto = np.array([
    np.nanmean(arr_tempo_resposta[arr_assunto == assunto])
    for assunto in top10_assuntos
])

# DataFrame resultado
df_top10_assuntos = pd.DataFrame({
    'Assunto': top10_assuntos,
    'Reclamações': top10_contagens_assunto,
    '% Total': pct_assuntos,
    'Nota Média': notas_por_assunto,
    'Taxa Resolução': taxa_resolucao_assunto,
    'Tempo Resposta (dias)': tempo_medio_assunto
})

display(
    df_top10_assuntos.style
    .format({
        '% Total': '{:.2f}%',
        'Reclamações': '{:,}',
        'Nota Média': '{:.2f}',
        'Taxa Resolução': '{:.1f}%',
        'Tempo Resposta (dias)': '{:.1f}'
    })
    .background_gradient(cmap='Blues', subset=['Reclamações'])
    .background_gradient(cmap='YlOrBr', subset=['Nota Média'], vmin=1, vmax=5)
    .background_gradient(cmap='Greens', subset=['Taxa Resolução'], vmin=0, vmax=100)
)

# Análise do assunto predominante
assunto_lider = top10_assuntos[0]
qtd_assunto_lider = top10_contagens_assunto[0]
pct_assunto_lider = pct_assuntos[0]
nota_assunto_lider = notas_por_assunto[0]

print(f"\nASSUNTO MAIS RECLAMADO:")
print(f"  {assunto_lider}")
print(f"  Volume: {qtd_assunto_lider:,} ({pct_assunto_lider:.1f}%)")
print(f"  Nota média: {nota_assunto_lider:.2f}")

# Assunto com pior nota
idx_pior_nota = np.nanargmin(notas_por_assunto)
print(f"\nASSUNTO COM PIOR SATISFAÇÃO:")
print(f"  {top10_assuntos[idx_pior_nota]}")
print(f"  Nota média: {notas_por_assunto[idx_pior_nota]:.2f}")

# Concentração top 5
top5_assuntos_concentracao = np.sum(pct_assuntos[:5])
print(f"\nTop 5 assuntos concentram: {top5_assuntos_concentracao:.1f}% das reclamações")

TOP 10 ASSUNTOS (TIPOS DE PROBLEMA) MAIS RECLAMADOS

Objetivo: Identificar as principais dores do consumidor brasileiro
--------------------------------------------------------------------------------


,Assunto,Reclamações,% Total,Nota Média,Taxa Resolução,Tempo Resposta (dias)
0,Cartão de Crédito / Cartão de Débito / Cartão de Loja,"462,077",30.06%,2.24,7.8%,6.0
1,Crédito Pessoal e Demais Empréstimos (exceto financiamento de imóveis e veículos),"303,461",19.74%,2.27,6.8%,6.0
2,Crédito Consignado / Cartão de Crédito Consignado / RMC (para beneficiários do INSS),"115,758",7.53%,2.00,4.2%,6.9
3,Consulta / Monitoramento de CPF,"92,866",6.04%,2.64,9.9%,3.6
4,Crédito Consignado (para servidores públicos ou trabalhadores do setor privado),"87,421",5.69%,2.24,6.9%,6.7
5,Conta corrente / Salário / Poupança /Conta Aposentadoria,"79,607",5.18%,2.30,10.0%,6.8
6,Financiamentos de Veículos / Leasing,"74,393",4.84%,2.20,5.5%,6.0
7,Serviço de pagamento online/ via celular/ maquininha,"61,589",4.01%,2.45,13.2%,6.6
8,Atendimento Bancário,"59,876",3.90%,2.29,8.2%,6.6
9,Score / Pontuação de Crédito,"44,287",2.88%,2.67,9.9%,3.5



ASSUNTO MAIS RECLAMADO:
  Cartão de Crédito / Cartão de Débito / Cartão de Loja
  Volume: 462,077 (30.1%)
  Nota média: 2.24

ASSUNTO COM PIOR SATISFAÇÃO:
  Crédito Consignado / Cartão de Crédito Consignado / RMC (para beneficiários do INSS)
  Nota média: 2.00

Top 5 assuntos concentram: 69.1% das reclamações


In [23]:
print("="*80)
print("TOP 10 CANAIS DE AQUISIÇÃO COM MAIS RECLAMAÇÕES")
print("="*80)
print("\nObjetivo: Identificar quais canais geram mais problemas e insatisfação")
print("-"*80)

# Extrai canais únicos e conta quantas reclamações cada um possui
canais_unicos, contagens_canal = np.unique(arr_produto, return_counts=True)

# Ordena do maior para o menor volume e seleciona os top 10
idx_canal_ord = np.argsort(contagens_canal)[::-1]
top10_canais = canais_unicos[idx_canal_ord][:10]
top10_contagens_canal = contagens_canal[idx_canal_ord][:10]

# Calcula o percentual de cada canal em relação ao total
pct_canais = (top10_contagens_canal / total_registros) * 100

# Calcula a nota média de satisfação por canal (1 a 5)
notas_por_canal = np.array([
    np.nanmean(arr_nota_completo[arr_produto == canal])
    for canal in top10_canais
])

# Calcula a taxa de resolução: % de casos resolvidos por canal
taxa_resolucao_canal = np.array([
    (np.sum((arr_produto == canal) & (arr_avaliacao == 'Resolvida')) / 
     np.sum(arr_produto == canal)) * 100
    for canal in top10_canais
])

# Calcula o tempo médio de resposta em dias por canal
tempo_medio_canal = np.array([
    np.nanmean(arr_tempo_resposta[arr_produto == canal])
    for canal in top10_canais
])

# Monta DataFrame com todos os indicadores
df_top10_canais = pd.DataFrame({
    'Canal de Aquisição': top10_canais,
    'Reclamações': top10_contagens_canal,
    '% Total': pct_canais,
    'Nota Média': notas_por_canal,
    'Taxa Resolução': taxa_resolucao_canal,
    'Tempo Resposta (dias)': tempo_medio_canal
})

# Exibe com formatação e gradientes de cor
display(
    df_top10_canais.style
    .format({
        '% Total': '{:.2f}%',
        'Reclamações': '{:,}',
        'Nota Média': '{:.2f}',
        'Taxa Resolução': '{:.1f}%',
        'Tempo Resposta (dias)': '{:.1f}'
    })
    .background_gradient(cmap='Blues', subset=['Reclamações'])
    .background_gradient(cmap='YlOrBr', subset=['Nota Média'], vmin=1, vmax=5)
    .background_gradient(cmap='Greens', subset=['Taxa Resolução'], vmin=0, vmax=100)
)

# Identifica o canal com maior volume de reclamações
canal_lider = top10_canais[0]
qtd_canal_lider = top10_contagens_canal[0]
pct_canal_lider = pct_canais[0]

print(f"\n🔴 CANAL COM MAIS RECLAMAÇÕES:")
print(f"  {canal_lider}")
print(f"  Volume: {qtd_canal_lider:,} ({pct_canal_lider:.1f}% do total)")
print(f"  Nota média: {notas_por_canal[0]:.2f}")

# Identifica o canal com pior satisfação (menor nota média)
idx_pior_canal = np.nanargmin(notas_por_canal)
print(f"\n⚠️  CANAL COM PIOR SATISFAÇÃO:")
print(f"  {top10_canais[idx_pior_canal]}")
print(f"  Nota média: {notas_por_canal[idx_pior_canal]:.2f}")
print(f"  Taxa de resolução: {taxa_resolucao_canal[idx_pior_canal]:.1f}%")

# Identifica o canal mais eficiente (maior taxa de resolução)
idx_melhor_canal = np.nanargmax(taxa_resolucao_canal)
print(f"\n✅ CANAL MAIS EFICIENTE (maior taxa de resolução):")
print(f"  {top10_canais[idx_melhor_canal]}")
print(f"  Taxa de resolução: {taxa_resolucao_canal[idx_melhor_canal]:.1f}%")
print(f"  Nota média: {notas_por_canal[idx_melhor_canal]:.2f}")

TOP 10 CANAIS DE AQUISIÇÃO COM MAIS RECLAMAÇÕES

Objetivo: Identificar quais canais geram mais problemas e insatisfação
--------------------------------------------------------------------------------


,Canal de Aquisição,Reclamações,% Total,Nota Média,Taxa Resolução,Tempo Resposta (dias)
0,Internet,"594,606",38.68%,2.30,8.6%,5.8
1,Não comprei / contratei,"396,547",25.80%,2.27,7.6%,6.0
2,Loja física,"308,350",20.06%,2.46,8.0%,6.4
3,Telefone,"192,331",12.51%,2.20,6.5%,6.0
4,SMS / Mensagem de texto,"15,755",1.02%,2.25,7.5%,5.7
5,Domicílio,"15,281",0.99%,2.18,5.8%,6.2
6,Catálogo,"7,903",0.51%,2.46,7.2%,5.4
7,"Stand, feiras e eventos","4,369",0.28%,2.51,9.9%,6.2
8,Ganhei de presente,"1,954",0.13%,2.52,12.4%,6.3



🔴 CANAL COM MAIS RECLAMAÇÕES:
  Internet
  Volume: 594,606 (38.7% do total)
  Nota média: 2.30

⚠️  CANAL COM PIOR SATISFAÇÃO:
  Domicílio
  Nota média: 2.18
  Taxa de resolução: 5.8%

✅ CANAL MAIS EFICIENTE (maior taxa de resolução):
  Ganhei de presente
  Taxa de resolução: 12.4%
  Nota média: 2.52
